# -- Description --

Forecast future household gas consumption for central heating from previous household gas usage data, then create an optimisation app which utilises the usage pattern from the future forecast to provide the homeowner the ability to apply a cost threshold to their gas consumption in order to have more control and aid in keeping household energy bills down. 

Cast as time series problem but also consider converting to a supervised problem to allow the usage of general machine learning models and also possibly neural networks.

Consider RMSE or MAE as possible performance metrics, reliant on data.

Use yearly data 2014-2018, to predict the year 2019.

# -- Library Imports --

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima.arima import auto_arima
from dython.nominal import correlation_ratio, theils_u
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import spearmanr, chi2_contingency, kruskal, anderson, normaltest, shapiro
from sklearn.preprocessing import LabelEncoder, StandardScaler, PowerTransformer, QuantileTransformer, PolynomialFeatures, MinMaxScaler
from pyearth import Earth
from sklearn.decomposition import PCA
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, VotingRegressor, StackingRegressor, BaggingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from scipy.stats import boxcox
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression , RFE
from sklearn.kernel_approximation import Nystroem
from ngboost import NGBRegressor
from ngboost.distns import Exponential
from hyperopt import atpe, tpe, fmin, hp, STATUS_OK, space_eval, Trials
from hyperopt.pyll.base import scope
from sklearn.model_selection import TimeSeriesSplit
from scipy.optimize import differential_evolution
from pygam.pygam import LinearGAM
from itertools import combinations, product
import joblib
from sklearn.utils import resample
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor
from kerastuner.tuners.bayesian import BayesianOptimization
from kerastuner.tuners.hyperband import Hyperband
from kerastuner.tuners.randomsearch import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import kerastuner as kt
import os
from pickle import dump, load
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
import dash_table

pd.set_option('display.max_columns', None)

In [ ]:
# requirements
"""
pandas==1.2.4
numpy==1.19.2
seaborn==0.11.1
matplotlib==3.3.4
statsmodels==0.12.2
scikit-learn==0.24.1
fbprophet==0.7.1
pmdarima==1.8.2
sweetviz==2.1.0
dython==0.6.3
scipy==1.6.2
sklearn-contrib-py-earth==0.1.0
xgboost==1.4.0
lightgbm==3.1.1
pycaret==2.2.0
ngboost==0.3.11
hyperopt==0.2.5
pygam==0.8.0
joblib==1.0.1
tensorflow==2.3.0
keras==2.4.3
kerastuner==1.0.1
"""

# -- Functions --

In [ ]:
def save_data(data, file_name):
    """ saves specified data as csv file with string filename """
    data.to_csv(file_name, encoding='utf-8', index=False)

In [ ]:
def load_train_data():
    """ loads csv data from string filename as pandas dataframe"""
    features = pd.read_csv('gas_consumption_monthly_train_features.csv')
    target = pd.read_csv('gas_consumption_monthly_train_target.csv')
    return features, target

In [ ]:
def load_test_data():
    """ loads csv data from string filename as pandas dataframe"""
    features = pd.read_csv('gas_consumption_monthly_test_features.csv')
    target = pd.read_csv('gas_consumption_monthly_test_target.csv')
    return features, target

In [ ]:
def load_clean_test_data():
    """ loads clean test data... best_features"""
    features, target = load_test_data()
    # remove features
    features = features[['month_sin', 'month_cos', 'exp_mean_ratio_outsidetemp_gas_used',
                         'exp_mean_ratio_housetemp_gas_used', 'knn_local_knowledge']]
    return features, target

In [ ]:
def load_clean_train_data():
    """ loads clean train data.. best features, removed data"""
    features, target = load_train_data()
    # remove features
    features = features[
        ['month_sin','month_cos','exp_mean_ratio_outsidetemp_gas_used',
         'exp_mean_ratio_housetemp_gas_used','knn_local_knowledge']] #,
    # remove 2013 entries
    features = features.iloc[2:,:].reset_index(drop=True)
    target = target[2:].reset_index(drop=True)
    return features, target

In [ ]:
def load_clean_train_data_lm():
    """ loads clean train data.. best features, removed data"""
    features, target = load_train_data()
    # remove features
    features = features[
        ['month_sin','month_cos','exp_mean_ratio_outsidetemp_gas_used','exp_mean_ratio_outsidetemp_gas_used_bxcx',
         'exp_mean_ratio_housetemp_gas_used','exp_mean_ratio_housetemp_gas_used_bxcx','knn_local_knowledge']] #'roll_mean_targ1'
    # remove 2013 entries
    features = features.iloc[2:,:].reset_index(drop=True)
    target = target[2:].reset_index(drop=True)
    return features, target

In [ ]:
def cost_of_gas(gas_used_m3, year):
    """ 
    returns string for the cost of gas used 
    params:
        gas_used_m3 = gas used in cubic meters
        year = year you want the cost for, for correct coefficient 
    """
    if year == 2013:
        year_coef = 9.76
    elif year == 2014:
        year_coef = 9.76
    elif year == 2015:
        year_coef = 9.74
    elif year == 2016:
        year_coef = 10.11
    elif year == 2017:
        year_coef = 10.1720
    elif year == 2018:
        year_coef = 11.1410
    elif year == 2019:
        year_coef = 11.3075 
    avg_cost_kwh = 3.8
    kwh = gas_used_m3 * year_coef
    cost_pence = kwh * avg_cost_kwh
    cost_pounds = np.round(cost_pence / 100, 2)
    return f'£{cost_pounds}'

In [ ]:
def expanding_window_cv(features, target, model, fold_size=1, init_fold=12, scoring='rmse'):
    """
    Performs cross validation for time series using an expanding window
    params:
        features = dataframe of features to perform cv on
        target = target of features to perform cv on
        model = model used to perform cv
        fold_size = size of folds to train and test on top of init_fold (number of rows)
        init_fold = size of first fold to train on (number of rows)
        score = model scoring metric, one of [rmse, mae]
    returns:
        cv model scores
    """
    rmse_scores = []
    mae_scores = []
    mape_scores = []
    r2_scores = []
    preds = []
    # inital cv fold
    train_feats, train_targ, test_feats, test_targ = features.iloc[0:init_fold,:], target[0:init_fold], features.iloc[init_fold:init_fold+fold_size,:], target[init_fold:init_fold+fold_size]
    model.fit(train_feats, train_targ)
    cv_pred = model.predict(test_feats)
    # get score
    #if scoring == 'rmse':
    rmse = mean_squared_error(test_targ, cv_pred, squared=False)
    #else:
    mae = mean_absolute_error(test_targ, cv_pred)
    mape = mean_absolute_percentage_error(test_targ, cv_pred)
    r2 = r2_score(test_targ, cv_pred)
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    mape_scores.append(mape)
    r2_scores.append(r2)
    preds.append(cv_pred)
    # calculate number of folds after initial fold
    num_folds = int((len(features)-(len(train_feats)+len(test_feats)))/fold_size)
    # cv on rest of folds
    for fold in range(num_folds):
        train_feats, train_targ, test_feats, test_targ = features.iloc[0:init_fold+(fold_size*(fold+1)),:], target[0:init_fold+(fold_size*(fold+1))], features.iloc[init_fold+(fold_size*(fold+1)):init_fold+((fold_size*(fold+1))+fold_size),:], target[init_fold+(fold_size*(fold+1)):init_fold+((fold_size*(fold+1))+fold_size)]
        model.fit(train_feats, train_targ)
        cv_pred = model.predict(test_feats)
        #if scoring == 'rmse':
        rmse = mean_squared_error(test_targ, cv_pred, squared=False)
        #else:
        mae = mean_absolute_error(test_targ, cv_pred)
        mape = mean_absolute_percentage_error(test_targ, cv_pred)
        r2 = r2_score(test_targ, cv_pred)
        rmse_scores.append(rmse)
        mae_scores.append(mae)
        mape_scores.append(mape)
        r2_scores.append(r2)
        preds.append(cv_pred)
    return rmse_scores, mae_scores, mape_scores, r2_scores, list(np.array(preds).flatten())

In [ ]:
# weighted average functions
def opt_ensemble_predictions(models, weights, features):
    """
    make ensemble predictions, returns prediction
    parameters:
        models = list of models/pipes to include in ensemble
        weights = array of weights to use for predictions
        features = features used for predictions

    returns:
        predicted values
    """
    # model predictions
    yhats = [model.predict(features) for model in models]
    yhats = np.array(yhats)
    # weighted sum across models.. predictions of the weighted models
    # summed products of predictions & weights
    weighted_preds = np.tensordot(yhats, weights, axes=((0), (0)))
    return weighted_preds

def opt_ensemble_eval(models, weights, features, target):
    """evaluates optimal weighted ensemble by averaging rmse"""
    yhats, _ = evaluate_ensemble(models, weights, features, target)
    eval_targ = target[12:]
    scores = []
    for i in range(4):
        rmse = mean_squared_error(eval_targ[(12*(i+1))-12:12*(i+1)], yhats[(12*(i+1))-12:12*(i+1)], squared=False)
        scores.append(rmse)
    return np.mean(scores)

def cv_pred(model, features, target):
    """ time series cross validation """
    _, _, _, _, preds = expanding_window_cv(features, target, model, fold_size=12, init_fold=12)
    return preds

def ensemble_predictions(models, weights, features, target):
    """
    make ensemble predictions, returns prediction
    parameters:
        models = list of models/pipes to include in ensemble
        weights = array of weights to use for predictions
        test_features = list of test features for each model included.. must be in same order

    returns:
        predicted classes, predicted probabilities for success, i.e label 1
    """
    # model predictions
    yhats = [cv_pred(models[i], features, target) for i in range(
        len(models))]
    yhats = np.array(yhats)
    # weighted sum across models.. predictions of the weighted models
    # summed products of predictions & weights
    weighted_preds = np.tensordot(yhats, weights, axes=((0), (0)))
    return weighted_preds


def evaluate_ensemble(models, weights, features, target):
    """returns predicted classes, success probabilities and metric evaluation for models"""
    # predictions
    yhat = ensemble_predictions(models, weights, features, target)
    # calculate fbeta loss
    return yhat, mean_squared_error(target[12:], yhat, squared=False)


def normalise(weights):
    """normalises weight vector to have unit norm, returns normalised weight vector"""
    # l1 vector norm
    result = np.linalg.norm(weights, 1)
    # check for vector of all 0's
    if result == 0.0:
        return weights
    return weights / result


def loss_function(weights, models, features, target):
    """loss function for optimisation, designed to be minimised"""
    # normalise weights
    norm_weights = normalise(weights)
    # calculate error
    _, score = evaluate_ensemble(
        models, norm_weights, features, target)
    return score

In [ ]:
def get_ml_models():
    models, names = list(), list()
    models.append(SVR())
    names.append('SVR')
    models.append(KNeighborsRegressor(n_jobs=-1))
    names.append('KNN')
    models.append(GradientBoostingRegressor(learning_rate=0.1, n_estimators=100, min_samples_split=3, max_features='auto', random_state=42))
    names.append('GB')
    models.append(AdaBoostRegressor(random_state=42))
    names.append('AB')
    models.append(ExtraTreesRegressor(n_estimators=100, max_depth=3, min_samples_split=3, max_features='auto', n_jobs=-1, random_state=42))
    names.append('ET')
    models.append(XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, min_child_weight=3, n_jobs=-1, random_state=42,))
    names.append('XGB')
    models.append(LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, min_child_samples=3, n_jobs=-1, random_state=42))
    names.append('LGBM')
    models.append(Earth())
    names.append('MARS')
    models.append(NGBRegressor(Dist=Exponential, n_estimators=100))
    names.append('NGB')
    models.append(LinearRegression(n_jobs=-1))
    names.append('LR')
    models.append(Ridge(random_state=42))
    names.append('R')
    models.append(SGDRegressor(shuffle=False, random_state=42))
    names.append('SGD')
    #models.append(LinearGAM(terms='s(0, n_splines=10)+s(1, n_splines=10)+s(2, n_splines=10)+s(3, n_splines=10)+l(4)'))
    #names.append('LGAM')
    return models, names

In [ ]:
class KeepColumnsTransformer():
    """ 
    custom transformer that keeps specified columns of dataframe
    """

    def __init__(self, columns=None):
        self.columns = columns

    def transform(self, X, **transform_params):
        if len(self.columns) == 1:
            drop_df = np.array(X[self.columns[0]].copy()).reshape(-1, 1)
        elif type(X) == pd.Series:
            drop_df = np.array(X[self.columns].copy()).reshape(1, -1)
        else:
            drop_df = X[self.columns].copy()
        return drop_df

    def fit(self, X, y=None, **fit_params):
        return self

# -- Supervised Gas Consumption Modelling --

## - Data Preperation

### Load Data

In [ ]:
# load gas usage data
gas_consumption = pd.read_csv('gas_consumption.csv')
outside_temp = pd.read_csv('outside_temp.csv')
# upstairs
bedroom1_temp = pd.read_csv('bedroom1_temp.csv')
bedroom2_temp = pd.read_csv('bedroom2_temp.csv')
bedroom3_temp = pd.read_csv('bedroom3_temp.csv')
bathroom_temp = pd.read_csv('bathroom_temp.csv')
desk_temp = pd.read_csv('desk_temp.csv')
# downstairs
living_room_temp = pd.read_csv('living_room_temp.csv')
kitchen_temp = pd.read_csv('kitchen_temp.csv')
storage_temp = pd.read_csv('storage_temp.csv')
# heating
upstairs_heating = pd.read_csv('heating_upstairs.csv')
downstairs_heating = pd.read_csv('heating_downstairs.csv')
# 

### Data Split Train/Test

In [ ]:
# remove last months of 2013 and early months of 2020
# split train/test
# gas consumption
gc_train = gas_consumption[gas_consumption['date_year'] < 2019]
gc_test = gas_consumption[(gas_consumption['date_year'] > 2018) & (
    gas_consumption['date_year'] < 2020)]
# outside temperature
ot_train = outside_temp[outside_temp['date_year'] < 2019]
ot_test = outside_temp[(outside_temp['date_year'] > 2018)
                       & (outside_temp['date_year'] < 2020)]
# bedroom 1 temperature
bedroom1_temp_train = bedroom1_temp[bedroom1_temp['date_year'] < 2019]
bedroom1_temp_test = bedroom1_temp[(bedroom1_temp['date_year'] > 2018) & (
    bedroom1_temp['date_year'] < 2020)]
# bedroom 2 temperature
bed2_train = bedroom2_temp[bedroom2_temp['date_year'] < 2019]
bed2_test = bedroom2_temp[(bedroom2_temp['date_year'] > 2018) & (
    bedroom2_temp['date_year'] < 2020)]
# bedroom 3 temperature
bed3_train = bedroom3_temp[bedroom3_temp['date_year'] < 2019]
bed3_test = bedroom3_temp[(bedroom3_temp['date_year'] > 2018) & (
    bedroom3_temp['date_year'] < 2020)]
# bathroom temperature
bath_train = bathroom_temp[bathroom_temp['date_year'] < 2019]
bath_test = bathroom_temp[(bathroom_temp['date_year'] > 2018) & (
    bathroom_temp['date_year'] < 2020)]
# desk temperature
desk_train = desk_temp[desk_temp['date_year'] < 2019]
desk_test = desk_temp[(desk_temp['date_year'] > 2018) &
                      (desk_temp['date_year'] < 2020)]
# living room temperature
lr_train = living_room_temp[living_room_temp['date_year'] < 2019]
lr_test = living_room_temp[(living_room_temp['date_year'] > 2018) & (
    living_room_temp['date_year'] < 2020)]
# kitchen temperature
k_train = kitchen_temp[kitchen_temp['date_year'] < 2019]
k_test = kitchen_temp[(kitchen_temp['date_year'] > 2018)
                      & (kitchen_temp['date_year'] < 2020)]
# storage temperature
st_train = storage_temp[storage_temp['date_year'] < 2019]
st_test = storage_temp[(storage_temp['date_year'] > 2018)
                       & (storage_temp['date_year'] < 2020)]

### Train Data

In [ ]:
# training data
# monthly measurements
# monthly avg temperature C
ot_monthly_train = ot_train.groupby(by=['date_year', 'date_month']).agg({
    'outside_temp_C': ['mean', 'median']})
# monthly consumption of gas per year
gc_monthly_train = gc_train.groupby(by=['date_year', 'date_month']).agg({
    'gas_consumed_01_M3': ['min', 'max']})
gc_monthly_train['gas_used'] = gc_monthly_train['gas_consumed_01_M3']['max'] - \
    gc_monthly_train['gas_consumed_01_M3']['min']
gc_monthly_train['gas_used_M3'] = gc_monthly_train['gas_used'] * 0.1
gc_monthly_train['avg_outside_tempC'] = np.round(
    ot_monthly_train['outside_temp_C']['mean'], 2)
# keep/remove columns
gc_monthly_train = gc_monthly_train[['avg_outside_tempC', 'gas_used_M3']]
# remove datetime index
gc_monthly_train.reset_index(inplace=True)
# season feature
gc_monthly_train['season'] = np.nan
# winter
w_idx1 = gc_monthly_train[(gc_monthly_train['date_month'] >= 1) & (
    gc_monthly_train['date_month'] <= 2)].index.tolist()
w_idx2 = gc_monthly_train[gc_monthly_train['date_month'] == 12].index.tolist()
winter_idx = w_idx1 + w_idx2
gc_monthly_train.loc[winter_idx, 'season'] = 'winter'
# spring
spring_idx = gc_monthly_train[(gc_monthly_train['date_month'] >= 3) & (
    gc_monthly_train['date_month'] <= 5)].index
gc_monthly_train.loc[spring_idx, 'season'] = 'spring'
# summer
summer_idx = gc_monthly_train[(gc_monthly_train['date_month'] >= 6) & (
    gc_monthly_train['date_month'] <= 8)].index
gc_monthly_train.loc[summer_idx, 'season'] = 'summer'
# autumn
autumn_idx = gc_monthly_train[(gc_monthly_train['date_month'] >= 9) & (
    gc_monthly_train['date_month'] <= 11)].index
gc_monthly_train.loc[autumn_idx, 'season'] = 'autumn'
# cyclical nature of month
gc_monthly_train['month_sin'] = np.sin(2*np.pi*gc_monthly_train.date_month/12)
gc_monthly_train['month_cos'] = np.cos(2*np.pi*gc_monthly_train.date_month/12)
# cylical nature of season
gc_monthly_train['season_label'] = 0
w_idx = gc_monthly_train[gc_monthly_train['season'] == 'winter'].index
gc_monthly_train.loc[w_idx, 'season_label'] = 1
sp_idx = gc_monthly_train[gc_monthly_train['season'] == 'spring'].index
gc_monthly_train.loc[sp_idx, 'season_label'] = 2
su_idx = gc_monthly_train[gc_monthly_train['season'] == 'summer'].index
gc_monthly_train.loc[su_idx, 'season_label'] = 3
a_idx = gc_monthly_train[gc_monthly_train['season'] == 'autumn'].index
gc_monthly_train.loc[a_idx, 'season_label'] = 4
gc_monthly_train['season_sin'] = np.sin(
    2*np.pi*gc_monthly_train.season_label/4)
gc_monthly_train['season_cos'] = np.cos(
    2*np.pi*gc_monthly_train.season_label/4)
gc_monthly_train.drop(columns=['season_label'], level=0, inplace=True)
gc_monthly_train = gc_monthly_train[
    ['date_year', 'date_month', 'month_sin', 'month_cos', 'season', 'season_sin', 'season_cos', 'avg_outside_tempC', 'gas_used_M3']]
gc_monthly_train

In [ ]:
# split & save train data
gc_monthly_train_features = gc_monthly_train[[
    'date_year', 'date_month', 'month_sin', 'month_cos', 'season', 'season_sin', 'season_cos', 'avg_outside_tempC']]
gc_monthly_train_target = gc_monthly_train['gas_used_M3']
save_data(gc_monthly_train_features,
          'gas_consumption_monthly_train_features.csv')  # save features
save_data(gc_monthly_train_target,
          'gas_consumption_monthly_train_target.csv')  # save target

Season feature created as it is fair to say seasonality plays a big part in the use of gas for central heating, more gas used in the winter period compared to the summer period, the cyclical nature of season was extracted into a feature aswell as for month. Also from the section of data above we can see that outside temperature affects gas usage, more gas looks to be used when the temperature outside is colder, except for the first observation which could be due to lack of data for this month.

### Test Data

In [ ]:
# monthly avg temperature C
ot_monthly_test = ot_test.groupby(by=['date_year', 'date_month']).agg({
    'outside_temp_C': ['mean', 'median']})
# monthly consumption of gas per year
gc_monthly_test = gc_test.groupby(by=['date_year', 'date_month']).agg({
    'gas_consumed_01_M3': ['min', 'max']})
gc_monthly_test['gas_used'] = gc_monthly_test['gas_consumed_01_M3']['max'] - \
    gc_monthly_test['gas_consumed_01_M3']['min']
gc_monthly_test['gas_used_M3'] = gc_monthly_test['gas_used'] * 0.1
gc_monthly_test['avg_outside_tempC'] = np.round(
    ot_monthly_test['outside_temp_C']['mean'], 2)
# keep/remove columns
gc_monthly_test = gc_monthly_test[['avg_outside_tempC', 'gas_used_M3']]
# remove datetime index
gc_monthly_test.reset_index(inplace=True)
# season feature
gc_monthly_train['season'] = np.nan
# winter
w_idx1 = gc_monthly_test[(gc_monthly_test['date_month'] >= 1) & (
    gc_monthly_test['date_month'] <= 2)].index.tolist()
w_idx2 = gc_monthly_test[gc_monthly_test['date_month'] == 12].index.tolist()
winter_idx = w_idx1 + w_idx2
gc_monthly_test.loc[winter_idx, 'season'] = 'winter'
# spring
spring_idx = gc_monthly_test[(gc_monthly_test['date_month'] >= 3) & (
    gc_monthly_test['date_month'] <= 5)].index
gc_monthly_test.loc[spring_idx, 'season'] = 'spring'
# summer
summer_idx = gc_monthly_test[(gc_monthly_test['date_month'] >= 6) & (
    gc_monthly_test['date_month'] <= 8)].index
gc_monthly_test.loc[summer_idx, 'season'] = 'summer'
# autumn
autumn_idx = gc_monthly_test[(gc_monthly_test['date_month'] >= 9) & (
    gc_monthly_test['date_month'] <= 11)].index
gc_monthly_test.loc[autumn_idx, 'season'] = 'autumn'
# cyclical nature of month
gc_monthly_test['month_sin'] = np.sin(2*np.pi*gc_monthly_test.date_month/12)
gc_monthly_test['month_cos'] = np.cos(2*np.pi*gc_monthly_test.date_month/12)
# cylical nature of season
gc_monthly_test['season_label'] = 0
w_idx = gc_monthly_test[gc_monthly_test['season'] == 'winter'].index
gc_monthly_test.loc[w_idx, 'season_label'] = 1
sp_idx = gc_monthly_test[gc_monthly_test['season'] == 'spring'].index
gc_monthly_test.loc[sp_idx, 'season_label'] = 2
su_idx = gc_monthly_test[gc_monthly_test['season'] == 'summer'].index
gc_monthly_test.loc[su_idx, 'season_label'] = 3
a_idx = gc_monthly_test[gc_monthly_test['season'] == 'autumn'].index
gc_monthly_test.loc[a_idx, 'season_label'] = 4
gc_monthly_test['season_sin'] = np.sin(2*np.pi*gc_monthly_test.season_label/4)
gc_monthly_test['season_cos'] = np.cos(2*np.pi*gc_monthly_test.season_label/4)
gc_monthly_test.drop(columns=['season_label'], level=0, inplace=True)
gc_monthly_test = gc_monthly_test[
    ['date_year', 'date_month', 'month_sin', 'month_cos', 'season', 'season_sin', 'season_cos', 'avg_outside_tempC', 'gas_used_M3']]
gc_monthly_test

In [ ]:
# split & save test data
gc_monthly_test_features = gc_monthly_test[[
    'date_year', 'date_month', 'month_sin', 'month_cos', 'season', 'season_sin', 'season_cos', 'avg_outside_tempC']]
gc_monthly_test_target = gc_monthly_test['gas_used_M3']
save_data(gc_monthly_test_features,
          'gas_consumption_monthly_test_features.csv')  # save features
save_data(gc_monthly_test_target,
          'gas_consumption_monthly_test_target.csv')  # save target

## - Data Cleansing -

### Nan Values

In [ ]:
mt_features, mt_target = load_train_data()
mt_features['gas_used_M3'] = mt_target
# analyse without lag nan values
monthly_train_features = mt_features[(mt_features['date_year']>=2015)&(mt_features['date_year']<=2018)]
monthly_train_features.info()

We have 62 observations and no nan values are present.

### Outliers

In [ ]:
# outliers
mt_features, mt_target = load_train_data()
mt_features['gas_used_M3'] = mt_target
# analyse without lag nan values
monthly_train_features = mt_features[(mt_features['date_year']>=2015)&(mt_features['date_year']<=2018)]
for col in monthly_train_features:
    # ony want to check outliers for temperature and gas used
    if monthly_train_features[col].dtype == 'float64' or monthly_train_features[col].dtype == 'int64':
        stats = monthly_train_features[col].describe()
        IQR = stats['75%'] - stats['25%']
        upper = stats['75%'] + 1.5 * IQR
        lower = stats['25%'] - 1.5 * IQR
        lower_bound_outliers = monthly_train_features[monthly_train_features[col] < lower]
        upper_bound_outliers = monthly_train_features[monthly_train_features[col] > upper]

        print('{}: {} upper outliers and {} lower outliers, bounds: upper bound = {}, lower bound = {}\n'.format(
            col, len(upper_bound_outliers), len(lower_bound_outliers), upper, lower))

Using the interquartile range, no outliers are present in the data.

In [ ]:
gc_monthly_train.describe(include='all')

No anomolous data can be seen so far, avg_outside_tempC looks as if it is fairly normally distributed with the mean and median values close together. gas_used_M3 seems to have a right skewed distribution.

## - EDA -

In [ ]:
monthly_train_features,_ = load_train_data()
monthly_train_features.head()

In [ ]:
# categorise features by data type
measured_cont = [
    'avg_outside_tempC', 'gas_used_M3', 'lag_12', 'lag_1', 'roll_mean_targ12', 'roll_mean_targ1', 'avg_tempC_lag12',
    'avg_tempC_lag1', 'tempC_exp_mean_per_month','gas_used_exp_mean_per_month','hodnhr_exp_mean_per_month',
    'avg_downstairs_tempC','avg_upstairs_tempC','hounhr_exp_mean_per_month','avg_dwn_temp_exp_mean_per_month',
    'avg_up_temp_exp_mean_per_month','avg_downstairs_temp_lag12','avg_downstairs_temp_lag1','avg_upstairs_temp_lag12',
    'avg_upstairs_temp_lag1','avg_house_tempC','avg_house_tempC_lag12','avg_house_tempC_lag1','exp_mean_ratio_outsidetemp_gas_used',
    'avg_house_temp_exp_mean_per_month','exp_mean_ratio_housetemp_gas_used','exp_mean_ratio_hodnhr_gas_used',
    'exp_mean_ratio_hounhr_gas_used','avg_tempC_lag12_power4','tempC_exp_mean_per_month_power4',
    'exp_mean_ratio_outsidetemp_gas_used_power4','exp_mean_ratio_housetemp_gas_used_power4','knn_local_knowledge',
    'roll_mean_targ1*mnth_sin','roll_mean_targ1*mnth_cos','exp_mean_ratio_outsidetemp_gas_used*mnth_sin',
    'exp_mean_ratio_outsidetemp_gas_used*mnth_cos','exp_mean_ratio_housetemp_gas_used*mnth_sin',
    'exp_mean_ratio_housetemp_gas_used*mnth_cos','knn_local_knowledge*mnth_sin','knn_local_knowledge*mnth_cos',
    'gas_used_exp_mean_per_season','roll_mean_targ1_lag_12']
discrete_cont = ['date_year', 'days_in_month','heat_on_dwn_num_hourly_recordings','heat_on_up_num_hourly_recordings',
                'hodnhr_lag12','hodnhr_lag1','hounhr_lag12','hounhr_lag1']
nominal = ['season', 'date_month', 'quarter']
cyclical = ['month_sin', 'month_cos', 'season_sin', 'season_cos', 'quarter_sin', 'quarter_cos']

### Distribution/Relationship Plots

In [ ]:
# feature distributions and target relationships
mt_features, mt_target = load_train_data()
mt_features['gas_used_M3'] = mt_target
# analyse without lag nan values
monthly_train_features = mt_features[(mt_features['date_year']>=2015)&(mt_features['date_year']<=2018)]

for col in monthly_train_features:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
    if col != 'gas_used_M3':
        if col not in nominal:
            if col in discrete_cont:
                sns.histplot(data=monthly_train_features, x=col,
                             stat='count', discrete=True, kde=True, ax=ax1)
            elif col in measured_cont:
                sns.histplot(data=monthly_train_features, x=col,
                             stat='count', bins=20, kde=True, ax=ax1)
            else:
                sns.histplot(data=monthly_train_features, x=col,
                             stat='count', bins=5, kde=True, ax=ax1)
            ax1.set_title(f'{col} Distribution')
            ax1.set_xlabel(f'{col}')
            ax1.set_ylabel('Count')
            ax1.grid(alpha=.3)

            sns.scatterplot(x=col, y='gas_used_M3',
                            data=monthly_train_features, ax=ax2)
            ax2.set_title(f'{col}/gas_used_M3 relationship')
            ax2.set_xlabel(f'{col}')
            ax2.set_ylabel('Gas Used (M3)')
        else:
            if col == 'season': # apply ordering
                monthly_train_features['season'] = monthly_train_features['season'].astype(
                    'category')
                monthly_train_features['season'] = monthly_train_features['season'].cat.reorder_categories(
                    ['winter', 'spring', 'summer', 'autumn'], ordered=True)

            sns.barplot(x=monthly_train_features[col].value_counts(
            ).index, y=monthly_train_features[col].value_counts().values, ax=ax1)
            ax1.set_title(f'{col} Distribution')
            ax1.set_xlabel(f'{col}')
            ax1.set_ylabel('Frequency')
            ax1.grid(alpha=.3)

            sns.boxplot(x=col, y='gas_used_M3',
                        data=monthly_train_features, ax=ax2)
            ax2.set_title(f'{col}/gas_used_M3 relationship')
            ax2.set_xlabel(f'{col}')
            ax2.set_ylabel('Gas Used (M3)')
    else:
        continue
    plt.show()

With the exception of the 2 months at the end of 2013, date_year and date_month have a uniform distribtution which is to be expected. date_month has a non linear relationship with gas usage, it shows seasonality where more gas is used in the winter months and hardly any used in the summer months. season has a uniform distribution with the exception of the 2 months which is as expected, it also confirms seasonality with gas usage, with more being used in the winter months and less being used in the summer months, spring and autumn show similar usage. avg_outside_tempC appears to be bimodal with a peak approx. around 6 dergrees C, with some digging the values around this peak are of winter months, the values around the other peak at approx. 18 have a seasonal mode of summer. There also appears to be a strong negative non linear relationship between gas usage and outside temperature which is to be expected. As the cyclical data is extracted from features with apparent relationships with gas usage they also will have relationships.

The relationship between quartely and gas_used_M3 also highlights that more gas is used in the colder months than the warmer months. We can see that more gas is used if a month has 31 days but this will be predominantly of the months december and january, it also highlights the gas used for february. Using the previous years values (lag 12) for gas_used_M3 we can see a fairly strong positive linear relationship compared to the previous month (lag 1) where there is not as strong a relationship.
We can see  that a rolling mean with a window of 12 previous months gas usage doesnt show any relationship with gas usage, but a rolling mean with a window of 2 previous months gas usage shows a strong positive linear relationship with gas usage. The previous years average temperature (avg_tempC_lag12) shows a fairly strong slight non linear negative relationship with gas usage compared to the previous months temperature which still shows a negative relationship but appears to be not as strong. The expanding mean for avg temperature for each month shows a strong non-linear negative relationship with gas usage with what appears to be a bimodal distribution, again this would be due to the temperature for colder and warmer months. The expanding mean for gas usage also shows a strong positive maybe slightly non linear relationship with gas usage.

In [ ]:
# target distribution
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['gas_used_M3'] = monthly_train_target
plt.figure(figsize=(10,6))
sns.histplot(x='gas_used_M3', data=monthly_train_features, bins=10, stat='count', kde=True)
plt.title('Gas Used Distribution')
plt.xlabel('Gas Used (M3)')
plt.ylabel('Count')
plt.show()

The target distribution shows a high number of observations approx. below 26 units, these observations are made up of mainly summer months with a few autumn and spring months, mainly may and september which are either side of the summer period and can be expected usage amounts for these months. Nothing appears unexpected.

### Normality Statistical Test

In [ ]:
# normality statistical test at 95% confidence
# H0 = feature comes from a normal distribution
# Ha = feature does not come from a normal distribution
# alpha = 5%
mt_features, _ = load_train_data()
gaussian = []
non_gaussian = []

for col in mt_features.columns:
    alpha = 0.05
    if col in measured_cont:
        print(f'\n{col}:')
        feat = mt_features[col].copy()
        shap_stat, shap_p = shapiro(feat)
        dagos_stat, dagos_p = normaltest(feat)
        stat_result = anderson(feat)

        count = 0
        if shap_p > alpha:
            count += 1
            print(
                f'Shapiro test:  Sample looks gaussian, stat = {shap_stat}, p = {shap_p}')
        else:
            print(
                f'Shapiro test:  Sample does not look gaussian, stat = {shap_stat}, p = {shap_p}')

        if dagos_p > alpha:
            count += 1
            print(
                f"D'agostino test:  Sample looks gaussian, stat = {dagos_stat}, p = {dagos_p}")
        else:
            print(
                f"D'agostino test:  Sample does not look gaussian, stat = {dagos_stat}, p = {dagos_p}")

        if stat_result.statistic < stat_result.critical_values[0]:
            count += 1
            print(f"Anderson test:  Sample looks gaussian")
        else:
            print(f"Anderson test:  Sample does not look gaussian")
        for i in range(len(stat_result.critical_values)):
            sl, cv = stat_result.significance_level[i], stat_result.critical_values[i]
            print(
                f"Anderson test:  stat = {stat_result.statistic}, sig = {sl}, crit val = {cv}")

        if count >= 2:
            gaussian.append(col)
        elif count <= 1:
            non_gaussian.append(col)

print(f'\nGaussian Features: {gaussian}')
print(f'\nNon Gaussian Features: {non_gaussian}')

### Interaction Plots

In [ ]:
# seasonal gas usage distributions
# remove last 2 months of 2013
mtf = monthly_train_features[(monthly_train_features['date_year'] >= 2014) & (
    monthly_train_features['date_year'] <= 2018)]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
plt.suptitle('Seasonal Distribution of Gas Usage')
sns.kdeplot(data=mtf, x='avg_outside_tempC',
            y='gas_used_M3', hue='season', ax=ax1)
ax1.grid(alpha=.2)
sns.histplot(data=mtf, x='gas_used_M3', hue='season', element='poly', ax=ax2)
ax2.grid(alpha=.2)
plt.show()

From plots above we can see the extent of gas usage for seasonality, winter occupies the higher values of gas usage, summer densely occupies the lower values of gas usage whilst there is a similar spread of usage for spring and autumn

In [ ]:
# monthly gas usage distributions
# remove last 2 months of 2013
mtf = monthly_train_features[(monthly_train_features['date_year'] >= 2014) & (
    monthly_train_features['date_year'] <= 2018)]
plt.figure(figsize=(14, 6))
plt.suptitle('Monthly Distribution of Gas Usage')
sns.scatterplot(data=mtf, x='date_month', y='gas_used_M3', hue='avg_outside_tempC',
                size='avg_outside_tempC', sizes=(20, 200), hue_norm=(0, 7), legend='brief', cmap='blue')
plt.grid(alpha=.2)
plt.show()

From the plot above, again we can say that outside temperature and month has a direct affect on household gas usage

In [ ]:
# proportion of gas usage per season and month per year
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['gas_used_M3'] = monthly_train_target

mtf = monthly_train_features[(monthly_train_features['date_year'] >= 2014) & (
    monthly_train_features['date_year'] <= 2018)]
# proportion of gas per season per year plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
plt.tight_layout(pad=4)
for yr in range(2014, 2019):
    yr_df = mtf[mtf['date_year'] == yr]
    yr_gas_total = np.round(yr_df['gas_used_M3'].sum(), 2)
    yr_props = []
    season = []
    for s in ['winter', 'spring', 'summer', 'autumn']:
        season.append(s)
        seas_gas_total = np.round(
            yr_df[yr_df['season'] == s]['gas_used_M3'].sum(), 2)
        gas_prop = np.round(seas_gas_total / yr_gas_total, 2)
        yr_props.append(gas_prop)
    sns.lineplot(x=season, y=yr_props, markers=True, label=yr, ax=ax1)
ax1.set_title('Propotion of Gas Used Per Season')
ax1.set_xlabel('Season')
ax1.set_ylabel('Proportion of Gas Used')
ax1.legend(loc='best')
ax1.grid(alpha=.2)
# proportion of gas per month per year
for yr in range(2014, 2019):
    yr_df = mtf[mtf['date_year'] == yr]
    yr_gas_total = np.round(yr_df['gas_used_M3'].sum(), 2)
    yr_props = []
    month = []
    for mnth in range(1, 13):
        month.append(mnth)
        month_gas_total = np.round(
            float(yr_df[yr_df['date_month'] == mnth]['gas_used_M3'].values), 2)
        gas_prop = np.round(month_gas_total / yr_gas_total, 2)
        yr_props.append(gas_prop)
    sns.lineplot(x=month, y=yr_props, markers=True, label=yr, ax=ax2)
ax2.set_title('Propotion of Gas Used Per Month')
ax2.set_xlabel('Month')
ax2.set_ylabel('Proportion of Gas Used')
ax2.legend(loc='best')
ax2.grid(alpha=.2)
plt.show()

As can be seen above the proportion of gas usage is fairly consistent across seasons, as for monthly proportions there seems to be some variability between february and march. This appears to be due to colder temperatures, 2017: feb-march = 5.78-9.32 degrees, 2018: feb-march = 0.08-5.49 degrees.

In [ ]:
# month temperature for year
monthly_train_features, monthly_train_target = load_train_data()

mtf = monthly_train_features[(monthly_train_features['date_year'] >= 2014) & (
    monthly_train_features['date_year'] <= 2018)]


years = sorted(mtf['date_year'].unique())
# proportion of gas per season per year plot
plt.figure(figsize=(14, 8))
for yr in years:
    yr_df = mtf[mtf['date_year'] == yr]
    mnth_temps = []
    months = sorted(mtf['date_month'].unique())
    for mnth in months:
        mnth_temp = np.round(
            float(yr_df[yr_df['date_month'] == mnth]['avg_outside_tempC'].values), 2)
        mnth_temps.append(mnth_temp)
    sns.lineplot(x=months, y=mnth_temps, markers=True, label=yr)
plt.title('Monthly Temperature C per Year')
plt.xlabel('Month')
plt.ylabel('Temperature')
plt.legend(loc='best')
plt.grid(alpha=.2)

### Correlational Heatmap

In [ ]:
# heatmap for colinearity
mt_features, mt_target = load_train_data()
mt_features['gas_used_M3'] = mt_target
# analyse without lag nan values
monthly_train_features = mt_features[(mt_features['date_year']>=2015)&(mt_features['date_year']<=2018)]

heatmap_dict = {}
columns = []

for col1 in monthly_train_features:
    if col1 not in cyclical:
        columns.append(col1)
        corr_vals = []
        xf = monthly_train_features[col1]
        for col2 in monthly_train_features:
            if col2 not in cyclical:
                yf = monthly_train_features[col2]
                if col1 in measured_cont and col2 in measured_cont:
                    score = spearmanr(xf, yf)[0]
                    corr_vals.append(score)
                if col1 in measured_cont and col2 in discrete_cont:
                    score = spearmanr(xf, yf)[0]
                    corr_vals.append(score)
                if col1 in measured_cont and col2 in nominal:
                    score = correlation_ratio(yf, xf)
                    corr_vals.append(score)
                if col1 in discrete_cont and col2 in discrete_cont:
                    score = spearmanr(xf, yf)[0]
                    corr_vals.append(score)
                if col1 in discrete_cont and col2 in measured_cont:
                    score = spearmanr(xf, yf)[0]
                    corr_vals.append(score)
                if col1 in discrete_cont and col2 in nominal:
                    score = correlation_ratio(yf, xf)
                    corr_vals.append(score)
                if col1 in nominal and col2 in nominal:
                    score = theils_u(xf, yf)
                    corr_vals.append(score)
                if col1 in nominal and col2 in measured_cont:
                    score = correlation_ratio(xf, yf)
                    corr_vals.append(score)
                if col1 in nominal and col2 in discrete_cont:
                    score = correlation_ratio(xf, yf)
                    corr_vals.append(score)
        heatmap_dict[col1] = corr_vals
corr_grid = pd.DataFrame(heatmap_dict, index=columns)
plt.figure(figsize=(30, 28))
plt.title('Correlation Heatmap')

sns.heatmap(corr_grid, vmin=-1, vmax=1, annot=True,
            cmap='RdBu_r', linewidths=0.3)
plt.yticks(rotation=45)
plt.show()

As Gaussian or gaussian like distributions are not present among measured and discrete features, the relationships between measured features, discrete features and combination of both features will be tested using spearmans rho coefficient as it does not have distributional assumptions. For relationships involving measured/discrete features and nominal features the correlation ratio will be used as it uses the levels of the nominal feature to determine relationship strength. For nominal/nominal relationships theils u coefficient will be used.

From the heatmap we can see that date_month, season & quarter show high association with gas usage, the correlation ratio says that given a gas usage measurement we are able to know which month, season or quarter it belongs to. We can see that date_year, days_in_month & roll_mean_lag12 have no correlation with gas usage and all other features have high to very high correlation with gas usage. It is clear we have high multicolinearity which is not a suprise as all features are derived from other features. With 95% confidence we can say that all relationships except date_year, days_in_month and roll_mean_lag12 with the target gas_used_M3 are all statistically significant. From this analysis we can drop date_year, days_in_month, roll_mean_lag12, gas usage with lag_1 as lag_12 is better correlated, avg_tempC_lag12 & lag1 as tempC_exp_mean_per_month is better correlated, also avg_outside_tempC as we will have to forecast this and the expanding mean will likely do the same job. As date_month shows the highest association with our target we could opt to to solely use this feature as to avoid multicolinearity but I feel dimensionality reduction via PCA could be a better option as interpretability is not vital. We can assess this further when we come to model feature selection.

***** After Feature Engineering *****
The top correlated/associated features that will be kept are; date_month (sin & cosine transformed version), exp_mean_ratio_outsidetemp_gas_used (the ratio of outside temperature and gas used expanding means), exp_mean_ratio_housetemp_gas_used (the ratio of house temperature and gas used expanding means) and knn_local_knowledge (knn model of 5 nearest neighbours). Although roll_mean_targ1 shows high correlation this feature will not be useful to use as rolling means are for short term forecasts (1 step ahead (in our case 1 month ahead)) and we are forecasting 12 steps (12 months) ahead.

### Statistical Significance

In [ ]:
# statistical significance tests between features and target
mt_features, mt_target = load_train_data()
mt_features['gas_used_M3'] = mt_target
# analyse without lag nan values
monthly_train_features = mt_features[(mt_features['date_year']>=2015)&(mt_features['date_year']<=2018)]
sig = []
non_sig = []
alpha = 0.05
print('With 95% confidence:\n')
for col in monthly_train_features:
    feat = monthly_train_features[col]
    target = monthly_train_features['gas_used_M3']
    if col in measured_cont:
        if col == 'gas_used_M3':
            continue
        else:
            spear_stat = spearmanr(feat, target.squeeze())
            if spear_stat[1] > alpha:
                print(
                    f'{col} is not statistically significant, relationship likely to have occured by chance\nSpearman correlation: {spear_stat[0]}, Pvalue : {spear_stat[1]}\n')
                non_sig.append(col)
            else:
                print(
                    f'{col} is statistically significant, relationship not likely to have occured by chance\nSpearman correlation: {spear_stat[0]}, Pvalue : {spear_stat[1]}\n')
                sig.append(col)
    if col in discrete_cont:
        kruskal_stat = kruskal(feat, target.squeeze())
        if kruskal_stat[1] > alpha:
            print(
                f'{col} is not statistically significant, relationship likely to have occured by chance\nKruskal statistic: {kruskal_stat[0]}, Pvalue : {kruskal_stat[1]}\n')
            non_sig.append(col)
        else:
            print(
                f'{col} is statistically significant, relationship not likely to have occured by chance\nKruskal statistic: {kruskal_stat[0]}, Pvalue : {kruskal_stat[1]}\n')
            sig.append(col)
    if col in nominal:
        le = LabelEncoder()
        feat_enc = le.fit_transform(feat)
        kruskal_stat = kruskal(feat_enc, target.squeeze())
        if kruskal_stat[1] > alpha:
            print(
                f'{col} is not statistically significant, relationship likely to have occured by chance\nKruskal statistic: {kruskal_stat[0]}, Pvalue : {kruskal_stat[1]}\n')
            non_sig.append(col)
        else:
            print(
                f'{col} is statistically significant, relationship not likely to have occured by chance\nKruskal statistic: {kruskal_stat[0]}, Pvalue : {kruskal_stat[1]}\n')
            sig.append(col)

print(f'\nSignificant:\n{sig}\n\nNon-significant:\n{non_sig}')

statistical siginificance hypothesis tests with significance level of 5% (confidence level of 95%), Ho = feature/target relationship likely to have occured by chance, Ha = feature/target relationship not likely to have occured bu chance.
Measured features tested using spearmans rho test, discrete features tested using kruskall-wallis test and nominal features also tested with kruskall-wallis test.

The majority of features, original and engineered, appear to have statistically significant relationships with the target gas used, significance suggests that the relationship between feature and target (if there is one) is not likely to have occured by chance, non-significance suggests the relationship (if there is one) is likely to have occured by chance. Features with non-significant relationships will be dropped as their relationships are likely to have occured by chance and therefore may effect generalisation on new data.

### Normality Plots

#### Top Features

In [ ]:
# normality plots and transforms for best features
monthly_train_features,_ = load_clean_train_data()
yj_pt = PowerTransformer(method='yeo-johnson')
bc_pt = PowerTransformer(method='box-cox')
qt = QuantileTransformer(
    n_quantiles=len(monthly_train_features), output_distribution='normal', random_state=1)

for col in monthly_train_features:
    fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(10,10))
    plt.suptitle(f'{col} normality qqplots')
    plt.tight_layout(pad=3)
    # original distribution
    qqplot(monthly_train_features[col], line='s', ax=ax1)
    ax1.set_title(f'{col} original')
    if monthly_train_features[col].min() <= 0:
        # yeo-johnson transform
        yj_trans = yj_pt.fit_transform(np.array(monthly_train_features[col]).reshape(-1,1))
        qqplot(yj_trans, line='s', ax=ax2)
        ax2.set_title(f'{col} yeo-johnson transform')
    else:
        # yeo-johnson transform
        yj_trans = yj_pt.fit_transform(np.array(monthly_train_features[col]).reshape(-1,1))
        qqplot(yj_trans, line='s', ax=ax2)
        ax2.set_title(f'{col} yeo-johnson transform')
        # boxcox transform
        bxcx_trans = bc_pt.fit_transform(np.array(monthly_train_features[col]).reshape(-1,1))
        qqplot(bxcx_trans, line='s', ax=ax3)
        ax3.set_title(f'{col} boxcox transform')
    # quantile transform
    quant_trans = qt.fit_transform(np.array(monthly_train_features[col]).reshape(-1,1))
    qqplot(quant_trans, line='s', ax=ax4)
    ax4.set_title(f'{col} quantile transform')

As the original features are non normally distributed and power transforms are unable to make them more gaussian like, we are not going to be able to infer results from linear models but as our focus is on forecasting future gas usage and not inference this will not effect us.

### Partial/Auto Correlation

In [ ]:
_, monthly_train_target = load_train_data()
plot_acf(monthly_train_target, lags=12)
plot_pacf(monthly_train_target, lags=12)

### PCA

#### Top Features

In [ ]:
# pca on top features
monthly_train_features, monthly_train_target = load_clean_train_data()
# remove 12 month lag nan values
monthly_train_features = monthly_train_features.iloc[12:,:]
sc = StandardScaler()
pca_df = sc.fit_transform(monthly_train_features)
# fit pca algorithm
pca = PCA(whiten=True, random_state=42).fit(pca_df)
# Plotting the Cumulative Summation of the Explained Variance
plt.figure(figsize=(10, 6))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)')  # for each component
plt.title('Explained Variance of Features')
plt.grid(alpha=0.3)
plt.show()

We can retain appox. 99% variance by reducing the data to 2 components

## - Feature Engineering -

### Train Features

In [ ]:
# create quarter and num days in month
monthly_train_features, monthly_train_target = load_train_data()
str_yr = monthly_train_features['date_year'].astype(int).astype(str)
str_mnth = monthly_train_features['date_month'].astype(int).astype(str)
str_date = str_yr.str.cat(str_mnth, sep='-')
str_date = pd.to_datetime(str_date)
monthly_train_features['quarter'] = str_date.dt.quarter
monthly_train_features['days_in_month'] = str_date.dt.days_in_month

In [ ]:
# target lag feature
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['lag_12'] = monthly_train_target.shift(12)
monthly_train_features['lag_1'] = monthly_train_target.shift(1)

In [ ]:
# target rolling mean
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['roll_mean_targ12'] = monthly_train_target.rolling(window=12).mean()
monthly_train_features['roll_mean_targ1'] = monthly_train_target.rolling(window=2).mean()

In [ ]:
# temperature lag feature
monthly_train_features, _ = load_train_data()
monthly_train_features['avg_tempC_lag12'] = monthly_train_features['avg_outside_tempC'].shift(12)
monthly_train_features['avg_tempC_lag1'] = monthly_train_features['avg_outside_tempC'].shift(1)

In [ ]:
# expanding average temperature of same previous months
monthly_train_features, _ = load_train_data()
monthly_train_features['tempC_exp_mean_per_month'] = 0
month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_temp = np.round(yr_df[yr_df['date_month']==mnth]['avg_outside_tempC'].values, 2)
        month_dict[mnth].append(mnth_temp)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0])
    # remove last value as will be for 2019
    month_dict[m].pop(-1)
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)
    
# insert expanding means
for mnth in sorted(monthly_train_features['date_month'].unique()):
    mnth_idx = monthly_train_features[monthly_train_features['date_month']==mnth].index.tolist()
    # remove 1st index if there are 6 indices, so we start from 2014
    if len(mnth_idx) == 6:
        mnth_idx.pop(0)
    for idx in range(len(mnth_idx)):
        monthly_train_features.loc[mnth_idx[idx], 'tempC_exp_mean_per_month'] = month_dict[mnth][idx]

In [ ]:
# expanding average gas usage per of same previous months
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['gas_used_M3'] = monthly_train_target
monthly_train_features['gas_used_exp_mean_per_month'] = 0
month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_gas = np.round(yr_df[yr_df['date_month']==mnth]['gas_used_M3'].values, 2)
        month_dict[mnth].append(mnth_gas)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0] + np.random.uniform(-2,5)) # add some randomness so its not exact
    # remove last value as will be for 2019
    month_dict[m].pop(-1)
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)
    
# insert expanding means
for mnth in sorted(monthly_train_features['date_month'].unique()):
    mnth_idx = monthly_train_features[monthly_train_features['date_month']==mnth].index.tolist()
    # remove 1st index if there are 6 indices, so we start from 2014
    if len(mnth_idx) == 6:
        mnth_idx.pop(0)
    for idx in range(len(mnth_idx)):
        monthly_train_features.loc[mnth_idx[idx], 'gas_used_exp_mean_per_month'] = month_dict[mnth][idx]
monthly_train_features.drop(columns=['gas_used_M3'], inplace=True)

In [ ]:
# cyclical nature of quarter
monthly_train_features, _ = load_train_data()

monthly_train_features['quarter_sin'] = np.sin(2*np.pi*monthly_train_features.quarter/4)
monthly_train_features['quarter_cos'] = np.cos(2*np.pi*monthly_train_features.quarter/4)

In [ ]:
# number of different hourly recordings of heating pump on per month
monthly_train_features, _ = load_train_data()

pump_on_dwn = downstairs_heating[downstairs_heating['heating_downstairs_on']==1]
daily_hours = pump_on_dwn.groupby(by=['date_year','date_month','date_day']).agg({'hour_':['nunique']})
daily_hours.reset_index(inplace=True)
daily_hours.columns = ['date_year','date_month','date_day','num_hourly_recordings_dwn']
daily_hours = daily_hours.groupby(by=['date_year','date_month']).agg({'num_hourly_recordings_dwn':['sum']})
daily_hours.reset_index(inplace=True)
daily_hours.columns = ['date_year','date_month','num_hourly_recordings_dwn']
daily_hours = daily_hours[daily_hours['date_year']<2019] # create train data

pump_on_up = upstairs_heating[upstairs_heating['heating_upstairs_on']==1]
daily_hours_up = pump_on_up.groupby(by=['date_year','date_month','date_day']).agg({'hour_':['nunique']})
daily_hours_up.reset_index(inplace=True)
daily_hours_up.columns = ['date_year','date_month','date_day','num_hourly_recordings_up']
daily_hours_up = daily_hours_up.groupby(by=['date_year','date_month']).agg({'num_hourly_recordings_up':['sum']})
daily_hours_up.reset_index(inplace=True)
daily_hours_up.columns = ['date_year','date_month','num_hourly_recordings_up']
daily_hours_up = daily_hours_up[daily_hours_up['date_year']<2019] # create train data

daily_hours['num_hourly_recordings_up'] = daily_hours_up['num_hourly_recordings_up']
monthly_train_features['heat_on_dwn_num_hourly_recordings'] = daily_hours['num_hourly_recordings_dwn']
monthly_train_features['heat_on_up_num_hourly_recordings'] = daily_hours['num_hourly_recordings_up']


In [ ]:
# expanding number of hourly recordings of heat downstairs per month of same previous months
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['hodnhr_exp_mean_per_month'] = 0
month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_recs = np.round(yr_df[yr_df['date_month']==mnth]['heat_on_dwn_num_hourly_recordings'].values, 2)
        month_dict[mnth].append(mnth_recs)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0] + np.random.uniform(-2,5)) # add some randomness so its not exact
    # remove last value as will be for 2019
    month_dict[m].pop(-1)
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)
    
# insert expanding means
for mnth in sorted(monthly_train_features['date_month'].unique()):
    mnth_idx = monthly_train_features[monthly_train_features['date_month']==mnth].index.tolist()
    # remove 1st index if there are 6 indices, so we start from 2014
    if len(mnth_idx) == 6:
        mnth_idx.pop(0)
    for idx in range(len(mnth_idx)):
        monthly_train_features.loc[mnth_idx[idx], 'hodnhr_exp_mean_per_month'] = month_dict[mnth][idx]

In [ ]:
# expanding number of hourly recordings of heat upstairs per month of same previous months
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['hounhr_exp_mean_per_month'] = 0
month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_recs = np.round(yr_df[yr_df['date_month']==mnth]['heat_on_up_num_hourly_recordings'].values, 2)
        month_dict[mnth].append(mnth_recs)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0] + np.random.uniform(-2,5)) # add some randomness so its not exact
    # remove last value as will be for 2019
    month_dict[m].pop(-1)
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)
    
# insert expanding means
for mnth in sorted(monthly_train_features['date_month'].unique()):
    mnth_idx = monthly_train_features[monthly_train_features['date_month']==mnth].index.tolist()
    # remove 1st index if there are 6 indices, so we start from 2014
    if len(mnth_idx) == 6:
        mnth_idx.pop(0)
    for idx in range(len(mnth_idx)):
        monthly_train_features.loc[mnth_idx[idx], 'hounhr_exp_mean_per_month'] = month_dict[mnth][idx]

In [ ]:
# avg downstairs temp
monthly_train_features, monthly_train_target = load_train_data()
lr = living_room_temp.groupby(by=['date_year','date_month']).agg({'living_temp_C':['mean']})
k = kitchen_temp.groupby(by=['date_year','date_month']).agg({'kitchen_temp_C':['mean']})
st = storage_temp.groupby(by=['date_year','date_month']).agg({'storage_temp_C':['mean']})
lr['kitchen_temp_C'] = k['kitchen_temp_C']['mean']
lr['storage_temp_C'] = st['storage_temp_C']['mean']
lr.reset_index(inplace=True)
# get average of rooms
avg_temp_dwn = []
for i in range(len(lr)):
    lt = lr['living_temp_C']['mean'][i]
    kt = lr['kitchen_temp_C'][i]
    st = lr['storage_temp_C'][i]
    mean_temp = np.mean([lt,kt,st])
    avg_temp_dwn.append(mean_temp)

lr['avg_downstairs_tempC'] = avg_temp_dwn
# create training data
lr = lr[lr['date_year']<2019]
lr = lr.iloc[1:,:].reset_index(drop=True)
monthly_train_features['avg_downstairs_tempC'] = np.round(lr['avg_downstairs_tempC'], 2)

In [ ]:
# avg upstairs temp
monthly_train_features, monthly_train_target = load_train_data()
b1 = bedroom1_temp.groupby(by=['date_year','date_month']).agg({'bedroom1_temp_C':['mean']})
b2 = bedroom2_temp.groupby(by=['date_year','date_month']).agg({'bedroom2_temp_C':['mean']})
b3 = bedroom3_temp.groupby(by=['date_year','date_month']).agg({'bedroom3_temp_C':['mean']})
bth = bathroom_temp.groupby(by=['date_year','date_month']).agg({'bathroom_temp_C':['mean']})
dsk = desk_temp.groupby(by=['date_year','date_month']).agg({'desk_temp_C':['mean']})
b1['bedroom2_temp_C'] = b2['bedroom2_temp_C']['mean']
b1['bedroom3_temp_C'] = b3['bedroom3_temp_C']['mean']
b1['bathroom_temp_C'] = bth['bathroom_temp_C']['mean']
b1['desk_temp_C'] = dsk['desk_temp_C']['mean']
b1.reset_index(inplace=True)
# get average of rooms
avg_temp_up = []
for i in range(len(b1)):
    b1t = b1['bedroom1_temp_C']['mean'][i]
    b2t = b1['bedroom2_temp_C'][i]
    b3t = b1['bedroom3_temp_C'][i]
    btht = b1['bathroom_temp_C'][i]
    dskt = b1['desk_temp_C'][i]
    mean_temp = np.mean([b1t,b2t,b3t,btht,dskt])
    avg_temp_up.append(mean_temp)

b1['avg_upstairs_tempC'] = avg_temp_up
# create training data
b1 = b1[b1['date_year']<2019]
b1 = b1.iloc[1:,:].reset_index(drop=True)
monthly_train_features['avg_upstairs_tempC'] = np.round(b1['avg_upstairs_tempC'], 2)

In [ ]:
# expanding mean of avg downstairs temp per month
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['avg_dwn_temp_exp_mean_per_month'] = 0
month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_temp = np.round(yr_df[yr_df['date_month']==mnth]['avg_downstairs_tempC'].values, 2)
        month_dict[mnth].append(mnth_temp)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0] + np.random.uniform(-2,5)) # add some randomness so its not exact
    # remove last value as will be for 2019
    month_dict[m].pop(-1)
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)
    
# insert expanding means
for mnth in sorted(monthly_train_features['date_month'].unique()):
    mnth_idx = monthly_train_features[monthly_train_features['date_month']==mnth].index.tolist()
    # remove 1st index if there are 6 indices, so we start from 2014
    if len(mnth_idx) == 6:
        mnth_idx.pop(0)
    for idx in range(len(mnth_idx)):
        monthly_train_features.loc[mnth_idx[idx], 'avg_dwn_temp_exp_mean_per_month'] = month_dict[mnth][idx]

In [ ]:
# expanding mean of avg upstairs temp per month
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['avg_up_temp_exp_mean_per_month'] = 0
month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_temp = np.round(yr_df[yr_df['date_month']==mnth]['avg_upstairs_tempC'].values, 2)
        month_dict[mnth].append(mnth_temp)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0] + np.random.uniform(-2,5)) # add some randomness so its not exact
    # remove last value as will be for 2019
    month_dict[m].pop(-1)
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)
    
# insert expanding means
for mnth in sorted(monthly_train_features['date_month'].unique()):
    mnth_idx = monthly_train_features[monthly_train_features['date_month']==mnth].index.tolist()
    # remove 1st index if there are 6 indices, so we start from 2014
    if len(mnth_idx) == 6:
        mnth_idx.pop(0)
    for idx in range(len(mnth_idx)):
        monthly_train_features.loc[mnth_idx[idx], 'avg_up_temp_exp_mean_per_month'] = month_dict[mnth][idx]

In [ ]:
# upstairs/downstairs heat on num hourly recordings lag features
monthly_train_features, _ = load_train_data()
monthly_train_features['hodnhr_lag12'] = monthly_train_features['heat_on_dwn_num_hourly_recordings'].shift(12)
monthly_train_features['hodnhr_lag1'] = monthly_train_features['heat_on_dwn_num_hourly_recordings'].shift(1)
monthly_train_features['hounhr_lag12'] = monthly_train_features['heat_on_up_num_hourly_recordings'].shift(12)
monthly_train_features['hounhr_lag1'] = monthly_train_features['heat_on_up_num_hourly_recordings'].shift(1)

In [ ]:
# avg upstairs/downstairs temp lag features
monthly_train_features, _ = load_train_data()
monthly_train_features['avg_downstairs_temp_lag12'] = monthly_train_features['avg_downstairs_tempC'].shift(12)
monthly_train_features['avg_downstairs_temp_lag1'] = monthly_train_features['avg_downstairs_tempC'].shift(1)
monthly_train_features['avg_upstairs_temp_lag12'] = monthly_train_features['avg_upstairs_tempC'].shift(12)
monthly_train_features['avg_upstairs_temp_lag1'] = monthly_train_features['avg_upstairs_tempC'].shift(1)

In [ ]:
# house avg temp, lag
monthly_train_features, _ = load_train_data()
monthly_train_features['avg_house_tempC'] = pd.Series(np.mean(np.stack((monthly_train_features.avg_downstairs_tempC,monthly_train_features.avg_upstairs_tempC)), axis=0))
monthly_train_features['avg_house_tempC_lag12'] = monthly_train_features['avg_house_tempC'].shift(12)
monthly_train_features['avg_house_tempC_lag1'] = monthly_train_features['avg_house_tempC'].shift(1)

In [ ]:
# expanding mean ratio of outside temp and gas used
monthly_train_features, _ = load_train_data()
monthly_train_features['exp_mean_ratio_outsidetemp_gas_used'] = (monthly_train_features['tempC_exp_mean_per_month']+1) /(monthly_train_features['gas_used_exp_mean_per_month']+1)

In [ ]:
# expanding mean of avg house temp per month
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['avg_house_temp_exp_mean_per_month'] = 0
month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_temp = np.round(yr_df[yr_df['date_month']==mnth]['avg_house_tempC'].values, 2)
        month_dict[mnth].append(mnth_temp)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0] + np.random.uniform(-2,5)) # add some randomness so its not exact
    # remove last value as will be for 2019
    month_dict[m].pop(-1)
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)
    
# insert expanding means
for mnth in sorted(monthly_train_features['date_month'].unique()):
    mnth_idx = monthly_train_features[monthly_train_features['date_month']==mnth].index.tolist()
    # remove 1st index if there are 6 indices, so we start from 2014
    if len(mnth_idx) == 6:
        mnth_idx.pop(0)
    for idx in range(len(mnth_idx)):
        monthly_train_features.loc[mnth_idx[idx], 'avg_house_temp_exp_mean_per_month'] = month_dict[mnth][idx]

In [ ]:
# expanding mean ratio of house temp and gas used
monthly_train_features, _ = load_train_data()
monthly_train_features['exp_mean_ratio_housetemp_gas_used'] = (monthly_train_features['avg_house_temp_exp_mean_per_month']+1) /(monthly_train_features['gas_used_exp_mean_per_month']+1)

In [ ]:
# expanding mean ratio heat on downstairs number of hourly recordings per month and gas used
monthly_train_features, _ = load_train_data()
monthly_train_features['exp_mean_ratio_hodnhr_gas_used'] = (monthly_train_features['hodnhr_exp_mean_per_month']+1) /(monthly_train_features['gas_used_exp_mean_per_month']+1)

In [ ]:
# expanding mean ratio heat on upstairs number of hourly recordings per month and gas used
monthly_train_features, _ = load_train_data()
monthly_train_features['exp_mean_ratio_hounhr_gas_used'] = (monthly_train_features['hounhr_exp_mean_per_month']+1) /(monthly_train_features['gas_used_exp_mean_per_month']+1)

In [ ]:
# knn feature engineering with best features 
monthly_train_features, monthly_train_target = load_train_data()
clean_cols = [
    'month_sin','month_cos','roll_mean_targ1','exp_mean_ratio_outsidetemp_gas_used','exp_mean_ratio_housetemp_gas_used']

knn = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
knn.fit(monthly_train_features[clean_cols].iloc[1:,:], monthly_train_target[1:])
pred = knn.predict(monthly_train_features[clean_cols].iloc[1:,:])

pred = np.insert(pred,0,0)
monthly_train_features['knn_local_knowledge'] = pred

In [ ]:
# month sin/cos combinations
monthly_train_features,_ = load_train_data()
monthly_train_features['roll_mean_targ1*mnth_sin'] = monthly_train_features['month_sin']*monthly_train_features['roll_mean_targ1']
monthly_train_features['roll_mean_targ1*mnth_cos'] = monthly_train_features['month_cos']*monthly_train_features['roll_mean_targ1']

monthly_train_features['exp_mean_ratio_outsidetemp_gas_used*mnth_sin'] = monthly_train_features['month_sin']*monthly_train_features['exp_mean_ratio_outsidetemp_gas_used']
monthly_train_features['exp_mean_ratio_outsidetemp_gas_used*mnth_cos'] = monthly_train_features['month_cos']*monthly_train_features['exp_mean_ratio_outsidetemp_gas_used']

monthly_train_features['exp_mean_ratio_housetemp_gas_used*mnth_sin'] = monthly_train_features['month_sin']*monthly_train_features['exp_mean_ratio_housetemp_gas_used']
monthly_train_features['exp_mean_ratio_housetemp_gas_used*mnth_cos'] = monthly_train_features['month_cos']*monthly_train_features['exp_mean_ratio_housetemp_gas_used']

monthly_train_features['knn_local_knowledge*mnth_sin'] = monthly_train_features['month_sin']*monthly_train_features['knn_local_knowledge']
monthly_train_features['knn_local_knowledge*mnth_cos'] = monthly_train_features['month_cos']*monthly_train_features['knn_local_knowledge']

In [ ]:
# expanding average gas usage per season
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['gas_used_M3'] = monthly_train_target
monthly_train_features['gas_used_exp_mean_per_season'] = 0
season_dict = {
    'winter': [],
    'spring': [],
    'summer': [],
    'autumn': [],
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for s in sorted(monthly_train_features['season'].unique()):
        s_gas = np.round(np.sum(yr_df[yr_df['season']==s]['gas_used_M3'].values), 2)
        season_dict[s].append(s_gas)
# remove empty array and expand mean
for s in season_dict:
    # remove 1st entry as not full season
    season_dict[s].pop(0)
    # insert 1st value twice then add randomness within standard deviation so we dont have a full season
    season_dict[s].insert(0, np.round(season_dict[s][0] + np.random.uniform(-np.std(season_dict[s]),np.std(season_dict[s])), 2)) # add some randomness so its not exact
    # remove last value as will be for 2019
    season_dict[s].pop(-1)
    # convert to pd series to use expanding
    season_dict[s] = pd.Series(season_dict[s])
    season_dict[s] = np.round(season_dict[s].expanding(1).mean(), 2)
    
# insert expanding means
for s in sorted(monthly_train_features['season'].unique()):
    for i in range(5):  
        yr_df = monthly_train_features[monthly_train_features['date_year']==2014+i]
        s_idx = yr_df[yr_df['season']==s].index.tolist()
        monthly_train_features.loc[s_idx, 'gas_used_exp_mean_per_season'] = season_dict[s][i]
monthly_train_features.drop(columns=['gas_used_M3'], inplace=True)

In [ ]:
### rolling target mean of 1 with lag of 12 months
monthly_train_features, monthly_train_target = load_train_data()
roll_targ = monthly_train_target.rolling(window=2).mean()
roll_lag = roll_targ.shift(12)
monthly_train_features['roll_mean_targ1_lag_12'] = roll_lag

In [ ]:
# boxcox transforms
monthly_train_features, monthly_train_target = load_train_data()
og = boxcox(monthly_train_features['exp_mean_ratio_outsidetemp_gas_used'], lmbda=0.04347125404618559)
hg = boxcox(monthly_train_features['exp_mean_ratio_housetemp_gas_used'], lmbda=-0.04065702816262846)

monthly_train_features['exp_mean_ratio_outsidetemp_gas_used_bxcx'] = og
monthly_train_features['exp_mean_ratio_housetemp_gas_used_bxcx'] = hg


In [ ]:
#save_data(monthly_train_features, 'gas_consumption_monthly_train_features.csv')

### Test Features

In [ ]:
# expanding average gas usage per of same previous months
monthly_test_features, monthly_test_target = load_test_data()
monthly_train_features, monthly_train_target = load_train_data()
monthly_train_features['gas_used_M3'] = monthly_train_target

month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_gas = np.round(yr_df[yr_df['date_month']==mnth]['gas_used_M3'].values, 2)
        month_dict[mnth].append(mnth_gas)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0] + np.random.uniform(-2,5)) # add some randomness so its not exact
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)

exp_mean_2019 = [month_dict[m][5] for m in month_dict]

monthly_test_features['gas_used_exp_mean_per_month'] = exp_mean_2019


In [ ]:
# expanding average temperature of same previous months
monthly_test_features, _ = load_test_data()
monthly_train_features, _ = load_train_data()

month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_temp = np.round(yr_df[yr_df['date_month']==mnth]['avg_outside_tempC'].values, 2)
        month_dict[mnth].append(mnth_temp)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0] + np.random.uniform(-np.std(month_dict[m]), np.std(month_dict[m])))
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)

exp_temp_2019 = [month_dict[m][5] for m in month_dict]

monthly_test_features['tempC_exp_mean_per_month'] = exp_temp_2019
    

In [ ]:
# expanding mean of avg house temp per month
monthly_test_features,_ = load_test_data()
monthly_train_features, monthly_train_target = load_train_data()

month_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}
for yr in sorted(monthly_train_features['date_year'].unique()):
    yr_df = monthly_train_features[monthly_train_features['date_year']==yr]
    for mnth in sorted(monthly_train_features['date_month'].unique()):
        mnth_temp = np.round(yr_df[yr_df['date_month']==mnth]['avg_house_tempC'].values, 2)
        month_dict[mnth].append(mnth_temp)
# remove empty array and expand mean
for m in month_dict:
    # remove empty array
    month_dict[m] = [x for x in month_dict[m] if x.size > 0]
    # insert 1st value twice so we dont have a nan
    if len(month_dict[m]) < 6:
        month_dict[m].insert(0, month_dict[m][0] + np.random.uniform(-np.std(month_dict[m]), np.std(month_dict[m]))) # add some randomness so its not exact
    # convert to pd series to use expanding
    month_dict[m] = pd.Series(month_dict[m])
    month_dict[m] = np.round(month_dict[m].expanding(1).mean(), 2)
    
exp_htemp_2019 = [month_dict[m][5] for m in month_dict]

monthly_test_features['avg_house_temp_exp_mean_per_month'] = exp_htemp_2019


In [ ]:
# expanding mean ratio of outside temp and gas usage
monthly_test_features, _ = load_test_data()
monthly_test_features['exp_mean_ratio_outsidetemp_gas_used'] = (monthly_test_features['tempC_exp_mean_per_month']) /(monthly_test_features['gas_used_exp_mean_per_month'])

In [ ]:
# expanding mean ratio of house temp and gas used
monthly_test_features, _ = load_test_data()
monthly_test_features['exp_mean_ratio_housetemp_gas_used'] = (monthly_test_features['avg_house_temp_exp_mean_per_month']) /(monthly_test_features['gas_used_exp_mean_per_month'])

In [ ]:
# knn feature engineering with best features 
monthly_test_features, _ = load_test_data()
monthly_train_features, monthly_train_target = load_train_data()
clean_cols = [
    'month_sin','month_cos','exp_mean_ratio_outsidetemp_gas_used','exp_mean_ratio_housetemp_gas_used'] #'roll_mean_targ1'

knn = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
knn.fit(monthly_train_features[clean_cols].iloc[1:,:], monthly_train_target[1:])
pred = knn.predict(monthly_test_features[clean_cols])

monthly_test_features['knn_local_knowledge'] = pred

In [ ]:
#save_data(monthly_test_features, 'gas_consumption_monthly_test_features.csv')

## - Baseline -

As we have no outliers present in the data we can use RMSE to evaluate our models, as mentioned we are unable to obtain gaussian  like distributed features, so our focus will be on tree based, support vector and boosting models.

In [ ]:
# target average baseline score full data from 2014
_, monthly_train_target = load_train_data()
baseline_rmse = monthly_train_target[2:].values.mean()
cost = cost_of_gas(baseline_rmse, 2019)
print(f'Baseline RMSE: {baseline_rmse},    Cost: {cost}')

In [ ]:
# baseline with cv -- full train data from 2014
monthly_train_features, monthly_train_target = load_clean_train_data()
monthly_train_features = monthly_train_features[['month_sin','month_cos']]
model = DecisionTreeRegressor(max_depth=3, min_samples_split=3, max_features='auto', random_state=42)
scores = expanding_window_cv(monthly_train_features, monthly_train_target, model, fold_size=12, init_fold=12, scoring='rmse')
avg_cost = cost_of_gas(np.mean(scores), 2019)
print(f'Simple model baseline\nAvg RMSE: {np.mean(scores)},   Std: {np.std(scores)}\nAvg Cost: {avg_cost}')

## - Modelling -

In [ ]:
# test models
models, names = get_ml_models()
mod_names = []
mod_rmse = []
mod_rmse_std = []
mod_mae = []
mod_mape = []
mod_r2 = []


for i in range(len(models)):
    if names[i] in ['SVR','MARS','LGAM']:
        pipe = Pipeline(steps =[
            ('sc', StandardScaler()),
            ('pca', PCA(n_components=2, whiten=True)),
            ('model', models[i])
        ])
        monthly_train_features, monthly_train_target = load_clean_train_data()
    elif names[i] in ['LR','R','SGD']:
        pipe = Pipeline(steps =[
            ('sc', StandardScaler()),
            ('pca', PCA(n_components=2, whiten=True)),
            ('model', models[i])
        ])
        monthly_train_features, monthly_train_target = load_clean_train_data_lm()
    else:
        pipe = Pipeline(steps=[
            ('sc', StandardScaler()),
            ('model', models[i])
        ])
        monthly_train_features, monthly_train_target = load_clean_train_data()
    rmse_scores, mae_scores, mape_scores, r2_scores, _ = expanding_window_cv(monthly_train_features, np.array(monthly_train_target).reshape(-1,), pipe, fold_size=12, init_fold=12, scoring='rmse')
    mod_names.append(names[i])
    mod_rmse.append(np.mean(rmse_scores))
    mod_rmse_std.append(np.std(rmse_scores))
    mod_mae.append(np.mean(mae_scores))
    mod_mape.append(np.mean(mape_scores))
    mod_r2.append(np.mean(r2_scores))

model_df = pd.DataFrame({'model': mod_names, 'Avg RMSE': mod_rmse, 'RMSE Std': mod_rmse_std, 'Avg MAE': mod_mae, 'Avg MAPE': mod_mape, 'Avg R2': mod_r2}).sort_values(by='Avg RMSE', ascending=True).reset_index(drop=True)
model_df

## - Feature Selection -

### Extra Trees

In [ ]:
# best subset extra trees
monthly_train_features, monthly_train_target = load_clean_train_data()
et_model = ExtraTreesRegressor(
    n_estimators=100, max_depth=3, min_samples_split=3, max_features='auto', n_jobs=-1, random_state=42)

pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', et_model)
    ]
)
best_combo = None
best_score = None
for i in range(5):
    print(f'Processing combinations of size {i+1}')
    combos = combinations(monthly_train_features.columns, i+1)
    for features in combos:
        mt_f = monthly_train_features[list(features)]
        rmse_scores, _, _, _, _ = expanding_window_cv(mt_f, np.array(monthly_train_target).reshape(-1,), pipe, fold_size=12, init_fold=12)
        avg_rmse = np.mean(rmse_scores)
        if best_score == None:
            best_score = avg_rmse
            best_combo = features
        elif best_score != None:
            if avg_rmse < best_score:
                best_score = avg_rmse
                best_combo = features

print(f'Extra Trees:\nBest RMSE: {best_score}\nBest Feature Combination: {best_combo}')

### Adaboost

In [ ]:
# best subset adaboost
monthly_train_features, monthly_train_target = load_clean_train_data()
ab_model = AdaBoostRegressor(random_state=42)

pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', ab_model)
    ]
)
best_combo = None
best_score = None
for i in range(5):
    print(f'Processing combinations of size {i+1}')
    combos = combinations(monthly_train_features.columns, i+1)
    for features in combos:
        mt_f = monthly_train_features[list(features)]
        rmse_scores, _, _, _, _ = expanding_window_cv(mt_f, np.array(monthly_train_target).reshape(-1,), pipe, fold_size=12, init_fold=12)
        avg_rmse = np.mean(rmse_scores)
        if best_score == None:
            best_score = avg_rmse
            best_combo = features
        elif best_score != None:
            if avg_rmse < best_score:
                best_score = avg_rmse
                best_combo = features

print(f'Adaboost:\nBest RMSE: {best_score}\nBest Feature Combination: {best_combo}')

### Gradient Boosting

In [ ]:
# best subset gradient boosting
monthly_train_features, monthly_train_target = load_clean_train_data()
gb_model = GradientBoostingRegressor(
    learning_rate=0.1, n_estimators=100, min_samples_split=3, max_features='auto', random_state=42)

pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', gb_model)
    ]
)
best_combo = None
best_score = None
for i in range(5):
    print(f'Processing combinations of size {i+1}')
    combos = combinations(monthly_train_features.columns, i+1)
    for features in combos:
        mt_f = monthly_train_features[list(features)]
        rmse_scores, _, _, _, _ = expanding_window_cv(mt_f, np.array(
            monthly_train_target).reshape(-1,), pipe, fold_size=12, init_fold=12)
        avg_rmse = np.mean(rmse_scores)
        if best_score == None:
            best_score = avg_rmse
            best_combo = features
        elif best_score != None:
            if avg_rmse < best_score:
                best_score = avg_rmse
                best_combo = features

print(
    f'Gradient Boosting:\nBest RMSE: {best_score}\nBest Feature Combination: {best_combo}')

## - Hyperparameter Tuning - 

### Extra Trees

In [ ]:
# extra trees regressor
monthly_train_features, monthly_train_target = load_clean_train_data()
monthly_train_features = monthly_train_features[['month_sin','month_cos','exp_mean_ratio_housetemp_gas_used', 'knn_local_knowledge']]

et_model = ExtraTreesRegressor(
    n_estimators=100, max_depth=3, min_samples_split=3, max_features='auto', n_jobs=-1, random_state=42)

pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', et_model)
    ]
)

def et_objective(params):
    pipe.set_params(**params)
    scores, _, _, _, _ = expanding_window_cv(
        monthly_train_features, np.array(monthly_train_target).reshape(-1,), pipe, fold_size=12, init_fold=12, scoring='rmse')
    avg_loss = np.mean(scores)
    return {'loss': avg_loss, 'params': params, 'status': STATUS_OK}

# define parameters
param_space = {
    'model__n_estimators': scope.int(hp.quniform('model__n_estimators', 50, 1000, 10)),
    'model__max_depth': scope.int(hp.quniform('model__max_depth', 1, 8, 1)),
    'model__min_samples_split': scope.int(hp.quniform('model__min_samples_split', 2, 12, 1)),
    'model__min_samples_leaf': scope.int(hp.quniform('model__min_samples_leaf', 1, 12, 1)),
    'model__max_features': hp.choice('model__max_features', ['auto','sqrt','log2']),
    'model__min_impurity_decrease': hp.uniform('model__min_impurity_decrease', 0.0, 0.6),
    'model__bootstrap': hp.choice('model__bootstrap', [True, False]),
    'model__ccp_alpha': hp.uniform('model__ccp_alpha', 0.0, 0.2)
}

# optimize model params
trials = Trials()
best_params = fmin(fn=et_objective, space=param_space,
                   algo=tpe.suggest, max_evals=1000, trials=trials)

best_param_space = space_eval(param_space, best_params)
print(f'Best Parameters:\n{best_param_space}')

In [ ]:
"""
all feats:
best loss: 16.453558344937093
Best Parameters:
{'model__bootstrap': False, 'model__ccp_alpha': 0.12885482457237743, 'model__max_depth': 4, 'model__max_features': 'log2', 
 'model__min_impurity_decrease': 0.49516117778118646, 'model__min_samples_leaf': 1, 'model__min_samples_split': 3, 
 'model__n_estimators': 100}
 
drop roll_mean_targ1----
best loss: 17.11262925718358
Best Parameters:
{'model__bootstrap': True, 'model__ccp_alpha': 0.11451043932660587, 'model__max_depth': 3, 'model__max_features': 'auto', 
 'model__min_impurity_decrease': 0.02605916245722535, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 
 'model__n_estimators': 770}
 
one hot encoded months----
best loss: 17.3384117745247
Best Parameters:
{'model__bootstrap': False, 'model__ccp_alpha': 0.14561048691123263, 'model__max_depth': 3, 'model__max_features': 'auto', 
 'model__min_impurity_decrease': 0.20006503985681634, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 
 'model__n_estimators': 500}

best features ------

"""

### Adaboost

In [ ]:
# adaboost
monthly_train_features, monthly_train_target = load_clean_train_data()
monthly_train_features= monthly_train_features[['month_sin', 'month_cos', 'knn_local_knowledge']]
ab_model = AdaBoostRegressor(random_state=42)

pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', ab_model)
    ]
)

def ab_objective(params):
    pipe.set_params(**params)
    scores, _, _, _, _ = expanding_window_cv(
        monthly_train_features, np.array(monthly_train_target).reshape(-1,), pipe, fold_size=12, init_fold=12, scoring='rmse')
    avg_loss = np.mean(scores)
    return {'loss': avg_loss, 'params': params, 'status': STATUS_OK}

# define parameters
param_space = {
    'model__n_estimators': scope.int(hp.quniform('model__n_estimators', 100, 1000, 10)),
    'model__learning_rate': hp.uniform('model__learning_rate', 0.5, 2.0),
    'model__loss': hp.choice('model__loss', ['linear','square','exponential']),
}

# optimize model params
trials = Trials()
best_params = fmin(fn=ab_objective, space=param_space,
                   algo=tpe.suggest, max_evals=1000, trials=trials)

best_param_space = space_eval(param_space, best_params)
print(f'Best Parameters:\n{best_param_space}')

In [ ]:
"""
best loss: 20.301556359196105
Best Parameters:
{'model__learning_rate': 1.5898277916109191, 'model__loss': 'square', 'model__n_estimators': 230}

best features-----

"""

### Gradient Boosting

In [ ]:
# gradient boosting
monthly_train_features, monthly_train_target = load_clean_train_data()
monthly_train_features= monthly_train_features[['month_cos', 'knn_local_knowledge']]
gb_model = GradientBoostingRegressor(
    learning_rate=0.1, n_estimators=100, min_samples_split=3, max_features='auto', random_state=42)

pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', gb_model)
    ]
)

def gb_objective(params):
    pipe.set_params(**params)
    scores, _, _, _, _ = expanding_window_cv(
        monthly_train_features, np.array(monthly_train_target).reshape(-1,), pipe, fold_size=12, init_fold=12, scoring='rmse')
    avg_loss = np.mean(scores)
    return {'loss': avg_loss, 'params': params, 'status': STATUS_OK}

# define parameters
param_space = {
    'model__n_estimators': scope.int(hp.quniform('model__n_estimators', 100, 1000, 10)),
    'model__learning_rate': hp.uniform('model__learning_rate', 0.001, 0.1),
    'model__max_depth': scope.int(hp.quniform('model__max_depth', 1, 8, 1)),
    'model__min_samples_split': scope.int(hp.quniform('model__min_samples_split', 2, 12, 1)),
    'model__min_samples_leaf': scope.int(hp.quniform('model__min_samples_leaf', 1, 12, 1)),
    'model__subsample': hp.uniform('model__subsample', 0.5, 0.9),
    'model__max_features': hp.choice('model__max_features', ['auto','sqrt','log2']),
    'model__min_impurity_decrease': hp.uniform('model__min_impurity_decrease', 0.0, 0.6),
    'model__loss': hp.choice('model__loss', ['ls','lad','huber','quantile']),
    'model__ccp_alpha': hp.uniform('model__ccp_alpha', 0.0, 0.2)
}

# optimize model params
trials = Trials()
best_params = fmin(fn=gb_objective, space=param_space,
                   algo=tpe.suggest, max_evals=1000, trials=trials)

best_param_space = space_eval(param_space, best_params)
print(f'Best Parameters:\n{best_param_space}')

Hyperparameter tuning summary:

    extra trees: RMSE = 16.453558344937093, avg error cost = £7.07

    gradient boosting: RMSE = 16.452704707642244, avg error cost = £7.07

    adaboost: RMSE = 19.687331563432124, avg error cost = £8.46

## - Model Ensembles -

### Voting Regressor

In [ ]:
monthly_train_features, monthly_train_target = load_clean_train_data()

et_model = ExtraTreesRegressor(
    bootstrap=False, ccp_alpha=0.15110530134134761, max_depth=3, max_features='auto', min_impurity_decrease=0.24850444259073817,
    min_samples_leaf=1, min_samples_split=2, n_estimators=80)

et_pipe = Pipeline(steps=[
    ('trans',  KeepColumnsTransformer(['month_sin','month_cos','exp_mean_ratio_housetemp_gas_used', 'knn_local_knowledge'])),
    ('sc', StandardScaler()),
    ('et', et_model)
])

gb_model = GradientBoostingRegressor(
    ccp_alpha=0.11899991337691343, learning_rate=0.08512323674014961, loss='lad', max_depth=1, max_features='auto', 
    min_impurity_decrease=0.3291504192766261, min_samples_leaf=3, min_samples_split=5, n_estimators=930, 
    subsample=0.6041295730833111)

gb_pipe = Pipeline(steps=[
    ('trans',  KeepColumnsTransformer(['month_cos', 'knn_local_knowledge'])),
    ('sc', StandardScaler()),
    ('xgb', gb_model)
])


estims = [('et_pipe', et_pipe), ('gb_pipe', gb_pipe)]
vc= VotingRegressor(
    estimators=estims,
    n_jobs=-1
)

mean_cv_score = []
mean_cv_std = []
for i in range(10):
    scores, _, _, _, _ = expanding_window_cv(
            monthly_train_features, np.array(monthly_train_target).reshape(-1,), vc, fold_size=12, init_fold=12)
    mean_cv_score.append(np.mean(scores))
    mean_cv_std.append(np.std(scores))

error_cost = cost_of_gas(np.mean(mean_cv_score), 2019)
print(f'Avg Rmse: {np.mean(mean_cv_score)},   Avg Std: {np.mean(mean_cv_std)},    Avg Error Cost: {error_cost}')

### Bagging Regressor

#### Extra Trees

In [ ]:
# extra trees bagging
monthly_train_features, monthy_train_target = load_clean_train_data()
et = ExtraTreesRegressor(
    bootstrap=False, ccp_alpha=0.15110530134134761, max_depth=3, max_features='auto', min_impurity_decrease=0.24850444259073817,
    min_samples_leaf=1, min_samples_split=2, n_estimators=80, n_jobs=-1, random_state=42)
bgr = BaggingRegressor(base_estimator=et, n_estimators=50, random_state=42, bootstrap=False)
bgr_pipe = Pipeline(steps=[
    ('trans',  KeepColumnsTransformer(['month_sin','month_cos','exp_mean_ratio_housetemp_gas_used', 'knn_local_knowledge'])),
    ('sc', StandardScaler()),
    ('model', bgr)
])
rmse, mae, mape, r2, _ = expanding_window_cv(monthly_train_features, np.array(monthly_train_target).reshape(-1,), bgr_pipe, fold_size=12, init_fold=12)
error_cost = cost_of_gas(np.mean(rmse), 2019)
print(f'Extra Trees Bagging:\nAvg RMSE: {np.mean(rmse)}, avg error cost: {error_cost}\nAvg MAE: {np.mean(mae)}\nAvg MAPE: {np.mean(mape)}\nAvg R2: {np.mean(r2)}')

#### Gradient Boosting

In [ ]:
monthly_train_features, monthy_train_target = load_clean_train_data()
gb_model = GradientBoostingRegressor(
    ccp_alpha=0.11899991337691343, learning_rate=0.08512323674014961, loss='lad', max_depth=1, max_features='auto', 
    min_impurity_decrease=0.3291504192766261, min_samples_leaf=3, min_samples_split=5, n_estimators=930, 
    subsample=0.6041295730833111, random_state=42)

bgr = BaggingRegressor(base_estimator=gb_model, n_estimators=50, random_state=42, bootstrap=False)
bgr_pipe = Pipeline(steps=[
    ('trans',  KeepColumnsTransformer(['month_cos', 'knn_local_knowledge'])),
    ('sc', StandardScaler()),
    ('model', bgr)
])
rmse, mae, mape, r2, _ = expanding_window_cv(monthly_train_features, np.array(monthly_train_target).reshape(-1,), bgr_pipe, fold_size=12, init_fold=12)
error_cost = cost_of_gas(np.mean(rmse), 2019)
print(f'Gradient Boosting Bagging:\nAvg RMSE: {np.mean(rmse)}, avg error cost: {error_cost}\nAvg MAE: {np.mean(mae)}\nAvg MAPE: {np.mean(mape)}\nAvg R2: {np.mean(r2)}')

### Optimised Weighted Regressor

In [ ]:
monthly_train_features, monthly_train_target = load_clean_train_data()
#tss = TimeSeriesSplit(n_splits = 4)

et_model = ExtraTreesRegressor(
    bootstrap=False, ccp_alpha=0.15110530134134761, max_depth=3, max_features='auto', min_impurity_decrease=0.24850444259073817,
    min_samples_leaf=1, min_samples_split=2, n_estimators=80, n_jobs=-1, random_state=42)

et_pipe = Pipeline(steps=[
    ('trans',  KeepColumnsTransformer(['month_sin','month_cos','exp_mean_ratio_housetemp_gas_used', 'knn_local_knowledge'])),
    ('sc', StandardScaler()),
    ('et', et_model)
])

gb_model = GradientBoostingRegressor(
    ccp_alpha=0.11899991337691343, learning_rate=0.08512323674014961, loss='lad', max_depth=1, max_features='auto', 
    min_impurity_decrease=0.3291504192766261, min_samples_leaf=3, min_samples_split=5, n_estimators=930, 
    subsample=0.6041295730833111, random_state=42)

gb_pipe = Pipeline(steps=[
    ('trans',  KeepColumnsTransformer(['month_cos', 'knn_local_knowledge'])),
    ('sc', StandardScaler()),
    ('gb', gb_model)
])

ensemble_pipes = [et_pipe, gb_pipe]
# bounds for weights
weight_bounds = [(0.0, 0.1) for _ in range(len(ensemble_pipes))]
# arguments for the loss function
search_arg = (ensemble_pipes, monthly_train_features, np.array(monthly_train_target).reshape(-1,))
# global optimization of ensemble weights
result = differential_evolution(
    loss_function, weight_bounds, search_arg, maxiter=100, tol=1e-7)
weights = normalise(result['x'])
print(f'weights: {weights}')
score = opt_ensemble_eval(
    ensemble_pipes, weights, monthly_train_features, np.array(monthly_train_target).reshape(-1,))
error_cost = cost_of_gas(score, 2019)
print(f'Avg optimised ensemble Rmse: {score}, avg error cost: {error_cost}')

## - Final Model -

In [ ]:
# load data
monthly_train_features, monthly_train_target = load_clean_train_data()
monthly_test_features, monthly_test_target = load_clean_test_data()
# load pipelines
# extra trees
et_model = ExtraTreesRegressor(
    bootstrap=False, ccp_alpha=0.15110530134134761, max_depth=3, max_features='auto', min_impurity_decrease=0.24850444259073817,
    min_samples_leaf=1, min_samples_split=2, n_estimators=80, n_jobs=-1, random_state=42)

et_pipe = Pipeline(steps=[
    ('trans',  KeepColumnsTransformer(['month_sin','month_cos','exp_mean_ratio_housetemp_gas_used', 'knn_local_knowledge'])),
    ('sc', StandardScaler()),
    ('et', et_model)
])
# gradient boosting
gb_model = GradientBoostingRegressor(
    ccp_alpha=0.11899991337691343, learning_rate=0.08512323674014961, loss='lad', max_depth=1, max_features='auto', 
    min_impurity_decrease=0.3291504192766261, min_samples_leaf=3, min_samples_split=5, n_estimators=930, 
    subsample=0.6041295730833111, random_state=42)

gb_pipe = Pipeline(steps=[
    ('trans',  KeepColumnsTransformer(['month_cos', 'knn_local_knowledge'])),
    ('sc', StandardScaler()),
    ('gb', gb_model)
])
# fit pipes
pipe_names = ['extra_trees', 'gradient_boosting']
ensemble_pipes = [et_pipe, gb_pipe]
ensemble_fit_pipes = [pipe.fit(monthly_train_features, np.array(monthly_train_target).reshape(-1,)) for pipe in ensemble_pipes]
# individual model predictions and save model
model_dict = {}
for i in range(len(ensemble_fit_pipes)):
    mod_preds = ensemble_fit_pipes[i].predict(monthly_test_features)
    mod_resids = monthly_test_target.values.flatten() - mod_preds
    mod_rmse = mean_squared_error(monthly_test_target, mod_preds, squared=False)
    mod_r2 = r2_score(monthly_test_target, mod_preds)
    print(f'{pipe_names[i]} model:     RMSE: {mod_rmse},    R2: {mod_r2}')
    joblib.dump(ensemble_fit_pipes[i], f'{pipe_names[i]}_home_win_model_pipeline.sav')
    model_dict[f'{pipe_names[i]} preds'] = mod_preds
    mod_df = pd.DataFrame({'Month': month,
                           '2019 Preds': mod_preds,
                           '2019 Actual': monthly_test_target,
                           'Residuals': mod_resids})
    mod_df.to_csv(f'{pipe_names[i]}_predictions_&_residuals_2019', encoding='utf-8', index=False)
weights = [0.54812394, 0.45187606] 
# save weights
np.savetxt('optimised_weighted_ensemble_weights.csv',
           np.array(weights), delimiter=',') 
# get ensemble predictions
ensemble_preds = opt_ensemble_predictions(ensemble_fit_pipes, weights, monthly_test_features) 
ens_residuals = monthly_test_target.values.flatten() - ensemble_preds 
ens_rmse = mean_squared_error(monthly_test_target, ensemble_preds, squared=False)
ens_r2 = r2_score(monthly_test_target, ensemble_preds)
print(f'Ensemble model:    RMSE: {rmse},    R2: {r2}')
# save predictions and residuals
month = ['January','February','March','April','May','June','July','August','September','October','November','December']
ensemble_df_2019 = pd.DataFrame({'Month': month,
                             '2019 Preds': ensemble_preds,
                             '2019 Actual': monthly_test_target,
                             'Residuals': residuals})
ensemble_df_2019.to_csv('ensemble_predictions_&_residuals_2019', encoding='utf-8', index=False)
# plot preds
plt.figure(figsize=(12,10))
plt.plot(range(1,13), monthly_test_target, label='actual', color='red')
plt.plot(range(1,13), ensemble_preds, label='ensemble', color='blue')
plt.plot(range(1,13), model_dict['extra_trees preds'], label='extra trees', color='green')
plt.plot(range(1,13), model_dict['gradient_boosting preds'], label='gradient boosting', color='orange')
plt.title('Model Predictions vs Actual')
plt.xlabel('Month')
plt.ylabel('Gas Used M3')
plt.legend(loc='best')
plt.grid(alpha=.3)
plt.show()

### Prediction Interval

In [ ]:
# prediction interval
# take each row including target (train data)
# draw a row (including target at random) (train data)
# store row and replace back
# repeat n times for bootstrap resample
# fit regression, predict new value (test data)
# take a single residual at random from original regression fit, add it to the predicted value and record result
# repeat say 1000 times
# find the 2.5th and 97.5th percentiles of the results for the prediction interval

monthly_train_features, monthly_train_target = load_clean_train_data()
monthly_test_features, monthly_test_target = load_clean_test_data()
# combine train features and target for resample
combi_train = pd.concat([monthly_train_features, monthly_train_target], axis=1)
# create best model
gb_model = GradientBoostingRegressor(
    ccp_alpha=0.11899991337691343, learning_rate=0.08512323674014961, loss='lad', max_depth=1, max_features='auto',
    min_impurity_decrease=0.3291504192766261, min_samples_leaf=3, min_samples_split=5, n_estimators=930,
    subsample=0.6041295730833111, random_state=42)

gb_pipe = Pipeline(steps=[
    ('trans',  KeepColumnsTransformer(['month_cos', 'knn_local_knowledge'])),
    ('sc', StandardScaler()),
    ('gb', gb_model)
])
# original prediction residuals
orig_resids = pd.read_csv('gradient_boosting_predictions_&_residuals_2019')[
    'Residuals']
# prediction intervals
percentiles = []
for i in range(len(monthly_test_features)):
    print(f'Processing prediction interval for prediction {i+1}')
    new_pred_vals = []
    for j in range(1000):
        bs_resamp = resample(combi_train, n_samples=30)  # bootstrap resample
        features = bs_resamp[
            ['month_sin', 'month_cos', 'exp_mean_ratio_outsidetemp_gas_used', 'exp_mean_ratio_housetemp_gas_used', 'knn_local_knowledge']]
        target = bs_resamp['gas_used_M3']
        gb_pipe.fit(features, np.array(target).reshape(-1,))
        pred = gb_pipe.predict(monthly_test_features.iloc[i,:])
        # check which residual to add, this fit or original fit
        rand_resid = np.random.choice(orig_resids)
        # add random original residual to prediction and store
        new_pred_vals.append(pred + rand_resid)
    p = np.percentile(new_pred_vals, q=[2.5, 97.5])  # compute percentiles
    percentiles.append(tuple(p))

# get percentiles
lower_perc = [x for x, _ in percentiles]
upper_perc = [x for _, x in percentiles]
# load model predictions
model_preds = pd.read_csv('gradient_boosting_predictions_&_residuals_2019')[
    '2019 Preds']

# plot prediction interval
plt.figure(figsize=(12, 10))
plt.plot(range(1, 13), model_preds, color='red', label='predictions')
plt.plot(range(1,13), monthly_test_target, color='blue', label='actual')
plt.plot(range(1,13), lower_perc, color='orange', label='2.5th percentile', linestyle='--')
plt.plot(range(1,13), upper_perc, color='orange', label='97.5th percentile')
plt.fill_between(range(1,13), lower_perc, upper_perc, alpha=.1, color='blue')
plt.title('Gradient Boosting Prediction Intervals (2.5, 97.5 percentiles)')
plt.xlabel('Month')
plt.ylabel('Gas Used M3')
plt.legend(loc='best')
plt.hlines(0, 1, 12, color='black', alpha=.5)
plt.show()

In [ ]:
lower_perc # 2.5th percentile

In [ ]:
upper_perc # 97.5th percentile

In [ ]:
# cost predictions, gradient boosting vs actual
preds = pd.read_csv('gradient_boosting_predictions_&_residuals_2019')['2019 Preds']
_, monthly_test_target = load_clean_test_data()
total_pred_gas = np.round(sum(preds), 2)
total_cost_gas = cost_of_gas(total_pred_gas, 2019)
total_actual_gas = float(np.round(sum(monthly_test_target.values), 2))
total_actual_cost = cost_of_gas(total_actual_gas, 2019)
print(f'total pred gas: {total_pred_gas},    total pred cost: {total_cost_gas}')
print(f'total actual gas: {total_actual_gas},    total actual cost: {total_actual_cost}')

## - Neural Network -

In [ ]:
# evaluate neural network
monthly_train_features, monthly_train_target = load_clean_train_data()

def nn():
    nn_model = Sequential(
        [
            Input(shape=(2,)),
            Dense(units=176, activation='relu', kernel_initializer='glorot_uniform', activity_regularizer=regularizers.l2(l2=0.0015525209418522444)),
            Dense(units=176, activation='relu', kernel_initializer='glorot_uniform'),
            Dropout(rate=0.34),
            Dense(units=400, activation='relu', kernel_initializer='glorot_uniform', activity_regularizer=regularizers.l2(l2=0.0015525209418522444)),
            Dense(units=1, activation='relu', kernel_initializer='glorot_uniform')
    ])

    loss=MeanSquaredError()
    opt=Adam(learning_rate=0.001989751755243559, beta_1=0.7500000000000001)
    nn_model.compile(optimizer=opt, loss=loss, metrics=[RootMeanSquaredError()])
    return nn_model

model = KerasRegressor(build_fn=nn, epochs=500, batch_size=12, shuffle=False)
nn_pipe = Pipeline(steps=[
    ('sc', StandardScaler()),
    ('pca', PCA(n_components=2, whiten=True)),
    ('model', model)
])
rmse_scores, mae_scores, mape_scores, r2_scores, _ = expanding_window_cv(monthly_train_features, np.array(monthly_train_target).reshape(-1,), nn_pipe, fold_size=12, init_fold=12)
print(f'Neural Network Scores:\nRMSE: {np.mean(rmse_scores)}\nMAE: {np.mean(mae_scores)}\nMAPE: {np.mean(mape_scores)}\nR2: {np.mean(r2_scores)}')

### Hyperparameter Tuning

In [ ]:
# neural network for evaluation, tuning
def nn_model(hp):
    d1 = hp.Int('units1', min_value=16, max_value=512, step=32)
    d2 = hp.Int('units2', min_value=16, max_value=512, step=32)
    d3 = hp.Int('units3', min_value=16, max_value=512, step=32)
    act1 = hp.Choice('activation1', values=['relu','tanh','elu','selu','exponential'])
    act2 = hp.Choice('activation2', values=['relu','tanh','elu','selu','exponential'])
    act3 = hp.Choice('activation3', values=['relu','tanh','elu','selu','exponential'])
    act4 = hp.Choice('activation4', values=['relu','tanh','elu','selu','exponential'])
    kern_init = hp.Choice('kernel_initializer', values=['glorot_uniform','glorot_normal','random_normal','random_uniform','truncated_normal','zeros','ones'])
    d_rate = hp.Float('drop_rate', min_value=0.05, max_value=0.4, step=0.01)
    l_2 = hp.Float('activity_regularizer', min_value=0.0001, max_value=0.01, sampling='log')
    l_rate = hp.Float('learning_rate', min_value=0.0001, max_value=0.01, sampling='log')
    b1 = hp.Float('beta', min_value=0.6, max_value=0.9, step=0.05)
    
    nn_model = Sequential(
        [
            Input(shape=(2,)),
            Dense(units=d1, activation=act1, kernel_initializer=kern_init, activity_regularizer=regularizers.l2(l2=l_2)),
            Dense(units=d2, activation=act2, kernel_initializer=kern_init),
            Dropout(rate=d_rate),
            Dense(units=d3, activation=act3, kernel_initializer=kern_init, activity_regularizer=regularizers.l2(l2=l_2)),
            Dense(units=1, activation=act4, kernel_initializer=kern_init)
    ])

    loss=MeanSquaredError()
    opt=Adam(learning_rate=l_rate, beta_1=b1)
    nn_model.compile(optimizer=opt, loss=loss, metrics=[RootMeanSquaredError()])
    return nn_model

In [ ]:
# keras tuner
monthly_train_features, monthly_train_target = load_clean_train_data()
# scale_features
sc = StandardScaler()
monthly_train_features = sc.fit_transform(monthly_train_features)
# apply pca
pca = PCA(n_components=2, whiten=True)
monthly_train_features = pca.fit_transform(monthly_train_features)

# tuner = BayesianOptimization(nn_model, objective=kt.Objective('root_mean_squared_error', direction='min'), max_trials=1000, num_initial_points=12, seed=42, directory=os.path.normpath('C:/nn_model_tuning'))
# tuner = Hyperband(nn_model, objective=kt.Objective('val_root_mean_squared_error', direction='min'), max_epochs=100, hyperband_iterations=1, seed=42, overwrite=True, directory=os.path.normpath('C:/'))
tuner = RandomSearch(nn_model, objective=kt.Objective('val_root_mean_squared_error', direction='min'), max_trials=500, seed=42, overwrite=True, directory=os.path.normpath('C:/'))
early_stop = EarlyStopping('val_root_mean_squared_error', patience=20)
tuner.search(monthly_train_features, np.array(monthly_train_target).reshape(-1,), epochs=500, callbacks=[early_stop], validation_split=.2)
best_hps = tuner.get_best_hyperparameters()[0]

In [ ]:
units1 = best_hps.get('units1')
units2 = best_hps.get('units2')
units3 = best_hps.get('units3')
activ1 = best_hps.get('activation1')
activ2 = best_hps.get('activation2')
activ3 = best_hps.get('activation3')
activ4 = best_hps.get('activation4')
k_init = best_hps.get('kernel_initializer')
d_rate = best_hps.get('drop_rate')
activ_reg = best_hps.get('activity_regularizer')
l_rate = best_hps.get('learning_rate')
beta1 = best_hps.get('beta')

print('Best Hyperparameters:\n')
print(f'Dense units 1: {units1}\nDense units 2: {units2}\nDense units 3: {units3}')
print(f'Activation 1: {activ1}\nActivation 2: {activ2}\nActivation 3: {activ3}\nActivation 4: {activ4}')
print(f'Kernel initializer: {k_init}\nDropout rate: {d_rate}')
print(f'Activity regularizer: {activ_reg}\nOpt learning rate: {l_rate}\nOpt beta 1: {beta1}')

### Final Model

In [ ]:
# train neural net
# load data
monthly_train_features, monthly_train_target = load_clean_train_data()
monthly_test_features, monthly_test_target = load_clean_test_data()
# create neural net
nn_model = Sequential(
    [
        Input(shape=(2,)),
        Dense(units=176, activation='selu', kernel_initializer='truncated_normal', activity_regularizer=regularizers.l2(l2=0.0015525209418522444)),
        Dense(units=176, activation='relu', kernel_initializer='truncated_normal'),
        Dropout(rate=0.34),
        Dense(units=400, activation='exponential', kernel_initializer='truncated_normal', activity_regularizer=regularizers.l2(l2=0.0015525209418522444)),
        Dense(units=1, activation='elu', kernel_initializer='truncated_normal')
])

loss=MeanSquaredError()
opt=Adam(learning_rate=0.001989751755243559, beta_1=0.7500000000000001)
nn_model.compile(optimizer=opt, loss=loss, metrics=[RootMeanSquaredError()])

# set up train & validation data.. validation data is year 2018
#monthly_train_features = monthly_train_features.iloc[:-12,:]
#monthly_train_target = monthly_train_target[:-12]
#monthly_val_features = monthly_train_features.iloc[-12:,:]
#monthly_val_target = monthly_train_target[-12:]

# data pipeline
sc = StandardScaler()
pca = PCA(n_components=2, whiten=True)
# fit scaler and pca on train data
sc.fit(monthly_train_features)
pca.fit(monthly_train_features)
monthly_train_features = sc.transform(monthly_train_features)
monthly_train_features = pca.transform(monthly_train_features)
# transform validation data with scaler and pca
monthly_test_features = sc.transform(monthly_test_features)
monthly_test_features = pca.transform(monthly_test_features)

# save scaler and pca
dump(sc, open('nn_scaler.pkl', 'wb')) ### to load later use... sc = load(open('nn_scaler.pkl', 'rb'))
dump(pca, open('nn_pca.pkl', 'wb')) ### to load later use... pca = load(open('nn_pca.pkl', 'rb'))

early_stop = EarlyStopping('val_root_mean_squared_error', patience=200)
######## saves full model #########  to load model... load_model('path/to/location')
checkpoint = ModelCheckpoint(filepath='neural_net_model.ckpt', monitor='val_root_mean_squared_error', save_weights_only=False, save_best_only=True, mode='min')
# fit neural net
nn_model.fit(monthly_train_features, np.array(monthly_train_target).reshape(-1,), epochs=10000, validation_data=(monthly_test_features, np.array(monthly_test_target).reshape(-1,)), callbacks=[early_stop, checkpoint], shuffle=False, batch_size=24)

In [ ]:
# predict and plot test data
# load data, scaler, pca and model
monthly_test_features, monthly_test_target = load_clean_test_data()
sc = load(open('nn_scaler.pkl', 'rb'))
pca = load(open('nn_pca.pkl', 'rb'))
nn = load_model('neural_net_model.ckpt')
# apply scaler and pca transforms
monthly_test_features = sc.transform(monthly_test_features)
monthly_test_features = pca.transform(monthly_test_features)
# get predictions and rmse
preds = nn.predict(monthly_test_features)
rmse = mean_squared_error(monthly_test_target, preds, squared=False)
print(f'Neural Net RMSE: {rmse}')
# cost of predicted gas compared to actual gas usage
total_pred_gas = np.round(float(sum(preds)),2)
total_cost_gas = cost_of_gas(total_pred_gas, 2019)
total_actual_gas = float(np.round(sum(monthly_test_target.values), 2))
total_actual_cost = cost_of_gas(total_actual_gas, 2019)
print(f'total pred gas: {total_pred_gas},    total pred cost: {total_cost_gas}')
print(f'total actual gas: {total_actual_gas},    total actual cost: {total_actual_cost}\n')
# plot pred vs actual
plt.figure(figsize=(12,10))
plt.plot(range(1,13), preds, label='predictions', color='blue')
plt.plot(range(1,13), monthly_test_target, label='actual', color='red')
plt.title('Neural Net predictions vs actual gas usage')
plt.xlabel('Month')
plt.ylabel('Gas Used M3')
plt.legend(loc='best')
plt.show()

### LSTM

In [ ]:
# split a multivariate sequence into samples
def split_sequences(features, target, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(features)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(features):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = features[i:end_ix, :], target[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
def lstm_model():
    model = Sequential([
        LSTM(units=64, activation='relu',
             return_sequences=True, input_shape=(12, 2), dropout=0.1, recurrent_dropout=0.1),
        LSTM(units=32, activation='relu'),
        Dense(units=12)
    ])
    loss = MeanSquaredError()
    opt = Adam()
    model.compile(optimizer=opt, loss=loss, metrics=[RootMeanSquaredError()])
    return model

In [ ]:
# load data
monthly_train_features, monthly_train_target = load_clean_train_data()
monthly_test_features, monthly_test_target = load_clean_test_data()
# load scaler and pca
sc = load(open('nn_scaler.pkl', 'rb'))
pca = load(open('nn_pca.pkl', 'rb'))
# transform train and test data
monthly_train_features = sc.transform(monthly_train_features)
monthly_train_features = pca.transform(monthly_train_features)
monthly_test_features = sc.transform(monthly_test_features)
monthly_test_features = pca.transform(monthly_test_features)
#monthly_test_features = monthly_test_features.reshape(-1,12,2)
# split train data
train_features, train_target = split_sequences(monthly_train_features, monthly_train_target, 12, 12)
# convert to features tensor
#train_features = tf.convert_to_tensor(train_features, dtype=tf.float32)
# load model
model = lstm_model()
# instantiate callbacks
early_stop = EarlyStopping('root_mean_squared_error', patience=100)
######## saves full model #########  to load model... load_model('path/to/location')
checkpoint = ModelCheckpoint(filepath='lstm_model.ckpt', monitor='root_mean_squared_error', save_weights_only=False, save_best_only=True, mode='min')

model.fit(train_features, train_target, epochs=10000, callbacks=[early_stop, checkpoint], shuffle=False)

In [ ]:
# predict and plot test data
# load data, scaler, pca and model
monthly_test_features, monthly_test_target = load_clean_test_data()
sc = load(open('nn_scaler.pkl', 'rb'))
pca = load(open('nn_pca.pkl', 'rb'))
lstm_model = load_model('lstm_model.ckpt')
# apply scaler and pca transforms
monthly_test_features = sc.transform(monthly_test_features)
monthly_test_features = pca.transform(monthly_test_features)
monthly_test_features = monthly_test_features.reshape(-1,12,2)
# get predictions and rmse
preds = lstm_model.predict(monthly_test_features).flatten()
rmse = mean_squared_error(monthly_test_target, preds, squared=False)
print(f'LSTM Model RMSE: {rmse}')
# cost of predicted gas compared to actual gas usage
total_pred_gas = np.round(float(sum(preds)),2)
total_cost_gas = cost_of_gas(total_pred_gas, 2019)
total_actual_gas = float(np.round(sum(monthly_test_target.values), 2))
total_actual_cost = cost_of_gas(total_actual_gas, 2019)
print(f'total pred gas: {total_pred_gas},    total pred cost: {total_cost_gas}')
print(f'total actual gas: {total_actual_gas},    total actual cost: {total_actual_cost}\n')
# plot pred vs actual
plt.figure(figsize=(12,10))
plt.plot(range(1,13), preds, label='predictions', color='blue')
plt.plot(range(1,13), monthly_test_target, label='actual', color='red')
plt.title('LSTM Model predictions vs actual gas usage')
plt.xlabel('Month')
plt.ylabel('Gas Used M3')
plt.legend(loc='best')
plt.show()

# -- Time Series Forecast --

## - Data Preperation -

### Load Data

In [ ]:
# load data
gas_consumption = pd.read_csv('gas_consumption.csv')

### Train/Test Split

In [ ]:
# remove last months of 2013 and early months of 2020
# split train/test
gc_train = gas_consumption[(gas_consumption['date_year'] > 2013) & (
    gas_consumption['date_year'] < 2019)]
gc_test = gas_consumption[(gas_consumption['date_year'] > 2018) & (
    gas_consumption['date_year'] < 2020)]

### Train Preparation

In [ ]:
# obtain monthly consumption of gas per year -- train data
gc_monthly = gc_train.groupby(by=['date_year', 'date_month']).agg({
    'gas_consumed_01_M3': ['min', 'max']})
gc_monthly['gas_used'] = gc_monthly['gas_consumed_01_M3']['max'] - \
    gc_monthly['gas_consumed_01_M3']['min']
gc_monthly['gas_used_M3'] = gc_monthly['gas_used'] * 0.1
gc_monthly.reset_index(inplace=True)
month_date = []
for i in range(len(gc_monthly)):
    yr = gc_monthly['date_year'][i]
    mnth = gc_monthly['date_month'][i]
    date_str = f'{yr}-{mnth}-01'
    month_date.append(date_str)
gc_monthly['month_date'] = month_date
gc_monthly['month_date'] = pd.to_datetime(gc_monthly['month_date'])
gc_monthly.index = gc_monthly['month_date']
gc_monthly.drop(columns=['date_year', 'date_month',
                'month_date'], level=0, inplace=True)
gc_monthly = gc_monthly[['gas_used_M3']]

### Test Preparation

In [ ]:
# testing data
# future monthly data to predict
gc_future_monthly = gc_test.groupby(by=['date_year', 'date_month']).agg({
    'gas_consumed_01_M3': ['min', 'max']})
gc_future_monthly['gas_used'] = gc_future_monthly['gas_consumed_01_M3']['max'] - \
    gc_future_monthly['gas_consumed_01_M3']['min']
gc_future_monthly['gas_used_M3'] = gc_future_monthly['gas_used'] * 0.1
# change the index to month datetime
gc_future_monthly.reset_index(inplace=True)
month_date = []
for i in range(len(gc_future_monthly)):
    yr = gc_future_monthly['date_year'][i]
    mnth = gc_future_monthly['date_month'][i]
    date_str = f'{yr}-{mnth}-01'
    month_date.append(date_str)
gc_future_monthly['month_date'] = month_date
gc_future_monthly['month_date'] = pd.to_datetime(
    gc_future_monthly['month_date'])
gc_future_monthly.index = gc_future_monthly['month_date']
gc_future_monthly.drop(
    columns=['date_year', 'date_month', 'month_date'], level=0, inplace=True)
gc_future_monthly = gc_future_monthly[['gas_used_M3']]

## - EDA -

In [ ]:
gc_monthly.describe()

In [ ]:
# autocorrelation plot
pd.plotting.autocorrelation_plot(gc_monthly)

### Stationarity

In [ ]:
gc_monthly.plot()

on first inspection trend looks fairly constant and annual seasonality is apparent. The amplitude of the cycles also seem fairly consistent

In [ ]:
# check mean & variance of splits from train data -- initial stationary test
split = np.int(len(gc_monthly) / 2)
split1, split2 = gc_monthly[:split], gc_monthly[split:]
mean1, mean2 = split1.mean(), split2.mean()
var1, var2 = split1.var(), split2.var()
print(f'mean 1: {mean1}, mean 2: {mean2}')
print(f'var 1: {var1}, var 2: {var2}')

initial stationary test appears to show relative stationarity with means and variances fairly consistent

In [ ]:
gc_monthly.hist()

### Decomposition

#### Original Decomposition

In [ ]:
# seasonal decomposition
decomp = seasonal_decompose(gc_monthly, model='additive')
decomp.plot()
plt.show()

In [ ]:
# test for stationarity
print('Results of Dickey-Fuller Test:')
st_test = adfuller(gc_monthly['gas_used_M3'], maxlag=12, autolag='AIC')
st_output = pd.Series(st_test[0:4], index=[
                     'Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in st_test[4].items():
    st_output['Critical Value (%s)' % key] = value
print(st_output)

As the test statistic is lower than the 1% critical value we can say with 99% confidence that the monthly gas consumption data is stationary, also the p-value is well below the significance level meaning we are not likely to have seen this by chance.

#### Seasonal Shifted Decomposition

In [ ]:
gc_shft_12 = gc_monthly['gas_used_M3'] - gc_monthly['gas_used_M3'].shift(12)
gc_shft_12 = gc_shft_12.dropna()
shft_decomp = seasonal_decompose(gc_shft_12, model='additive')
shft_decomp.plot()
plt.show()

In [ ]:
# test for stationarity on shifted data
print('Results of Dickey-Fuller Test:')
st_test = adfuller(gc_shft_12, maxlag=12, autolag='AIC')
st_output = pd.Series(st_test[0:4], index=[
                     'Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in st_test[4].items():
    st_output['Critical Value (%s)' % key] = value
print(st_output)

Although the seasonally shifted data still appears to be stationary, the test statitistic after seasonal shifted decomposition is just below the 1% critical value and the p-value is a lot higher than the original decomposition, meaning the original has better stationarity than the seasonally shifted data.

In [ ]:
gc_shft_1 = gc_monthly['gas_used_M3'] - gc_monthly['gas_used_M3'].shift(1)
gc_shft_1 = gc_shft_1.dropna()
shft_decomp = seasonal_decompose(gc_shft_1, model='additive')
shft_decomp.plot()
plt.show()

In [ ]:
# test for stationarity on shifted data
print('Results of Dickey-Fuller Test:')
st_test = adfuller(gc_shft_1, maxlag=12, autolag='AIC')
st_output = pd.Series(st_test[0:4], index=[
                     'Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in st_test[4].items():
    st_output['Critical Value (%s)' % key] = value
print(st_output)

## - Baseline -

In [ ]:
# use the average of monthly averages for each month
avg_monthly = gc_monthly.copy()
avg_monthly.reset_index(inplace=True)
total_avg_gas = []
for i in range(12):
    mnth_idx = avg_monthly['month_date'].dt.month == i+1
    avg_mnth_gas = np.round(
        np.mean(avg_monthly[mnth_idx]['gas_used_M3'].values), 2)
    total_avg_gas.append(avg_mnth_gas)

rmse = mean_squared_error(gc_future_monthly, total_avg_gas, squared=False)

print(
    f'Monthly average baseline RMSE score: {rmse}')

# plot of average gas forecast
gc_future_pred = pd.DataFrame(
    {'avg_pred': total_avg_gas}, index=gc_future_monthly.index)
plt.figure(figsize=(10, 8))
plt.plot(gc_future_monthly.index,
         gc_future_monthly['gas_used_M3'].values, color='blue', label='actual')
plt.plot(gc_future_pred['avg_pred'], color='red', label='monthly average pred')
plt.legend(loc='best')
plt.title('Monthly Gas Forecast Using Monthly Averages')
plt.xlabel('Forecast Horizon (Months)')
plt.ylabel('Gas Used (M3)')
plt.show()

## - Modelling -

### Polynomial Model

In [ ]:
# model seasonality
X = [i%12 for i in range(0, len(gc_monthly))]
y = gc_monthly.values.flatten()
degree = 9
coef = np.polyfit(X, y, degree)
print(f'Coefficients: {coef}')
# create curve
curve = list()
for i in range(len(X)):
    value = coef[-1]
    for d in range(degree):
        value += X[i]**(degree-d) * coef[d]
    curve.append(value)
# plot curve over original data
plt.figure(figsize=(10,8))
plt.plot(gc_monthly.values)
plt.plot(curve, color='red', linewidth=3)
plt.show()

In [ ]:
data_model = pd.Series(curve[:12], index=gc_future_monthly.index)
rmse = mean_squared_error(gc_future_monthly.values, data_model.values, squared=False)
r2 = r2_score(gc_future_monthly.values, data_model.values)
print(f'Poly Model RMSE: {rmse},   R2: {r2}')
plt.figure(figsize=(10,8))
plt.plot(gc_future_monthly, label='actual', color='blue')
plt.plot(data_model, label='model', color='red')
plt.title('Polynomial Model vs Actual Data')
plt.xlabel('Month')
plt.ylabel('Gas USed M3')
plt.legend()
plt.show()

### ARIMA

In [ ]:
arima = ARIMA(gc_monthly, order=(2,0,5), seasonal_order=(0,0,0,12))
arima_fit = arima.fit()
print(arima_fit.summary())
# line plot of residuals
resids = pd.DataFrame(arima_fit.resid)
resids.plot()
plt.show()
# density plot of residuals
resids.plot(kind='kde')
plt.show()
# summary stats of residuals
print(resids.describe())

In [ ]:
arima = ARIMA(gc_monthly, order=(2,0,5), seasonal_order=(0,0,0,12))
arima_fit = arima.fit()
arima_preds = arima_fit.predict(start=len(gc_monthly)+1, end=len(gc_monthly)+12, typ='levels')
# evaluate forecasts
rmse = mean_squared_error(gc_future_monthly.values, arima_preds, squared=False)
print('Test RMSE: %.3f' % rmse)
# plot forecasts against actual outcomes
plt.figure(figsize=(10,8))
plt.plot(range(1,13), gc_future_monthly.values, label='actual', color='blue')
plt.plot(range(1,13), arima_preds, label='predicted', color='red')
plt.legend()
plt.show()

### Prophet

In [ ]:
# prophet forecast
# train data
train_data = pd.DataFrame(
    {'gas_used_M3': gc_monthly['gas_used_M3'].values}, index=gc_monthly.index)
train_data.reset_index(inplace=True)
train_data.columns = ['ds', 'y']
# test data
test_data = gc_future_monthly.reset_index()
test_data = test_data[['month_date']]
test_data.columns = ['ds']

model = Prophet(changepoint_prior_scale=0.001, seasonality_prior_scale=0.1)
model.fit(train_data)
forecast = model.predict(test_data)
actual = gc_future_monthly['gas_used_M3'].values
predicted = forecast['yhat'].values
predicted_upper = forecast['yhat_upper'].values
predicted_lower = forecast['yhat_lower'].values

proph_rmse = mean_squared_error(actual, predicted, squared=False)
proph_r2 = r2_score(actual, predicted)
print(
    f'Prophet forecast RMSE score: {proph_rmse},    R2 score: {proph_r2}')

pred_df = pd.DataFrame({'pred': predicted}, index=gc_future_monthly.index)
plt.figure(figsize=(12, 8))
plt.plot(gc_future_monthly.index,
         gc_future_monthly['gas_used_M3'], color='blue', label='actual')
plt.plot(pred_df, color='red', label='prophet forecast')
#plt.fill_between(pred_df['pred'], predicted_lower, predicted_upper, color='red', alpha=.2)
plt.legend(loc='best')
plt.title('Prophet Forecast of HouseHold Gas Usage')
plt.xlabel('Forecast Horizon (Months)')
plt.ylabel('Gas Used (M3)')
plt.show()

In [ ]:
# prophet hyperparameter tuning

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    model = Prophet(**params).fit(train_data)  # Fit model with given params
    model_cv = cross_validation(model, period=12, horizon=12, parallel="processes")
    model_p = performance_metrics(model_cv, rolling_window=1)
    rmses.append(model_p['rmse'].values[0])

# Find the best parameters
#tuning_results = pd.DataFrame(all_params)
#tuning_results['rmse'] = rmses
#print(tuning_results)

best_params = all_params[np.argmin(rmses)]
print(f'Top parameters:\n{best_params}')

In [ ]:
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
tuning_results.sort_values(by='rmse').reset_index()

## - Final Models Plot -

In [ ]:
# final time series plot
plt.figure(figsize=(10,8))
plt.plot(range(1,13), data_model.values, label='poly prediction', color='red')
plt.plot(range(1,13), arima_preds, label='arima prediction', color='orange')
plt.plot(range(1,13), pred_df.values, label='prophet prediction', color='green')
plt.plot(range(1,13), gc_future_monthly.values, label='actual', color='blue')
plt.title('Time Series Model Predictions vs Actual')
plt.xlabel('Month')
plt.ylabel('Gas Used M3')
plt.legend()
plt.grid(alpha=.3)
plt.show()